# Filtering the Data

In [1]:
import pandas as pd
df_100_clean = pd.read_csv('route 100 clean.csv')
df_100_clean_25jan = df_100_clean[df_100_clean['service_date'].str.contains('2025-01-25')]

def filter_and_export_by_time(df, start_hour, end_hour):
    # Make a copy to avoid modifying original df
    df_filtered = df.copy()

    # Convert 'scheduled' and 'actual' to datetime if not already
    df_filtered['scheduled'] = pd.to_datetime(df_filtered['scheduled'])
    df_filtered['actual'] = pd.to_datetime(df_filtered['actual'])

    # Filter based on the scheduled hour
    df_filtered = df_filtered[
        (df_filtered['scheduled'].dt.hour >= start_hour) &
        (df_filtered['scheduled'].dt.hour <= end_hour)
    ]

    # Format 'scheduled' and 'actual' to HH:MM:SS
    df_filtered['scheduled'] = df_filtered['scheduled'].dt.strftime('%H:%M:%S')
    df_filtered['actual'] = df_filtered['actual'].dt.strftime('%H:%M:%S')

    # Define output filename
    filename = f"filtered_route100_{start_hour}h_{end_hour}h.csv"

    # Export to CSV
    df_filtered.to_csv(filename, index=False)
    print(f"File saved as {filename}")

    # Return the filtered dataframe
    return df_filtered

# Filter between 8am and 8pm
#df_100_8am_8pm = filter_and_export_by_time(df_100_clean_25jan, 8, 20)
#df_100_12pm_8pm = filter_and_export_by_time(df_100_clean_25jan, 12, 20)
df_100_4pm_8pm = filter_and_export_by_time(df_100_clean_25jan, 16, 20)
#df_100_3pm_8pm = filter_and_export_by_time(df_100_clean_25jan, 15, 20)


File saved as filtered_route100_16h_20h.csv


# 1. Using PuLP solver

## a. 3 hours

In [3]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import pulp as pl
import time

# Define bus operation parameters
BUS_PARAMS = {
    'avg_speed_mph': 12,  # Average bus speed in mph
    'cost_per_mile': 2.5,  # Operating cost per mile
    'cost_per_minute': 1.0,  # Cost per minute of operation
    'waiting_cost_factor': 0.5,  # Cost factor for waiting time
    'delay_risk_factor': 2.0  # Cost factor for delay risk
}

# Define the depot
DEPOT_ID = 'fellsway_garage'

# Define distances from Google Maps screenshots (in miles)
DISTANCES = {
    'depot_to_welst': 2.0,  # Fellsway Garage to Wellington St
    'depot_to_elmst': 1.4,  # Fellsway Garage to Elm St
    'welst_to_depot': 2.0,  # Wellington St to Fellsway Garage
    'elmst_to_depot': 1.4,  # Elm St to Fellsway Garage
    'elmst_to_welst': 1.5,  # Estimated distance between stops
    'welst_to_elmst': 1.5   # Estimated distance between stops
}

def calculate_travel_time(distance, speed=BUS_PARAMS['avg_speed_mph']):
    """Calculate travel time based on distance and speed"""
    return (distance / speed) * 60

def create_trip_nodes(csv_path):
    """Create trip nodes from the filtered CSV data"""
    # Read the filtered data
    df = pd.read_csv(csv_path)
    
    # Print data summary
    print(f"Total records: {len(df)}")
    print(f"Unique half_trip_ids: {df['half_trip_id'].nunique()}")
    print(f"Unique stop_ids: {df['stop_id'].unique()}")
    print(f"Unique time_point_ids: {df['time_point_id'].unique()}")
    
    # Map time_point_ids to our location codes
    location_map = {
        'elmst': 'elmst',
        'welst': 'welst'
    }
    
    # Group by half_trip_id
    trips = []
    
    # For each half_trip_id, find its startpoint and endpoint
    for half_trip_id, group in df.groupby('half_trip_id'):
        startpoints = group[group['point_type'] == 'Startpoint']
        endpoints = group[group['point_type'] == 'Endpoint']
        
        if not startpoints.empty and not endpoints.empty:
            # Get start and end locations
            start_time_point = startpoints['time_point_id'].iloc[0]
            end_time_point = endpoints['time_point_id'].iloc[0]
            
            # Map to our location codes
            start_location = location_map.get(start_time_point, start_time_point)
            end_location = location_map.get(end_time_point, end_time_point)
            
            trip = {
                'half_trip_id': half_trip_id,
                'route_id': group['route_id'].iloc[0],
                'direction_id': group['direction_id'].iloc[0],
                'start_location': start_location,
                'end_location': end_location,
                'scheduled_start_time': startpoints['scheduled'].iloc[0],
                'scheduled_end_time': endpoints['scheduled'].iloc[0],
                'actual_start_time': startpoints['actual'].iloc[0],
                'actual_end_time': endpoints['actual'].iloc[0]
            }
            trips.append(trip)
    
    # Convert to DataFrame
    trips_df = pd.DataFrame(trips)
    
    # Parse datetime strings
    for col in ['scheduled_start_time', 'scheduled_end_time', 'actual_start_time', 'actual_end_time']:
        if col in trips_df.columns:
            # Convert times to datetime objects
            trips_df[col] = pd.to_datetime(trips_df[col], errors='coerce')
    
    return trips_df

def calculate_costs(trips_df, depot_id=DEPOT_ID):
    """
    Calculate costs between trips and depot connections using distances from Google Maps
    
    Args:
        trips_df: DataFrame of trip nodes
        depot_id: ID of the depot
        
    Returns:
        DataFrame with trip-to-trip costs and depot connections
    """
    # Get parameters
    cost_per_mile = BUS_PARAMS['cost_per_mile']
    cost_per_minute = BUS_PARAMS['cost_per_minute']
    waiting_cost_factor = BUS_PARAMS['waiting_cost_factor']
    delay_risk_factor = BUS_PARAMS['delay_risk_factor']
    
    # Create empty lists to store connections
    connections = []
    
    # Add depot to start connections
    for _, trip in trips_df.iterrows():
        # Get distance from depot to this trip's start location
        if trip['start_location'] == 'welst':
            distance = DISTANCES['depot_to_welst']
        elif trip['start_location'] == 'elmst':
            distance = DISTANCES['depot_to_elmst']
        else:
            # Use average distance for any other locations
            distance = (DISTANCES['depot_to_welst'] + DISTANCES['depot_to_elmst']) / 2
            
        # Calculate travel time
        travel_time = calculate_travel_time(distance)
        
        # Calculate costs
        deadheading_cost = distance * cost_per_mile + travel_time * cost_per_minute
        
        connection = {
            'from_id': depot_id,
            'to_id': trip['half_trip_id'],
            'from_type': 'depot',
            'to_type': 'trip',
            'from_location': depot_id,
            'to_location': trip['start_location'],
            'distance_miles': distance,
            'travel_time_minutes': travel_time,
            'deadheading_cost': deadheading_cost,
            'waiting_cost': 0,
            'delay_risk_cost': 0,
            'total_cost': deadheading_cost
        }
        connections.append(connection)
    
    # Add end to depot connections
    for _, trip in trips_df.iterrows():
        # Get distance from this trip's end location to depot
        if trip['end_location'] == 'welst':
            distance = DISTANCES['welst_to_depot']
        elif trip['end_location'] == 'elmst':
            distance = DISTANCES['elmst_to_depot']
        else:
            # Use average distance for any other locations
            distance = (DISTANCES['welst_to_depot'] + DISTANCES['elmst_to_depot']) / 2
            
        # Calculate travel time
        travel_time = calculate_travel_time(distance)
        
        # Calculate costs
        deadheading_cost = distance * cost_per_mile + travel_time * cost_per_minute
        
        connection = {
            'from_id': trip['half_trip_id'],
            'to_id': depot_id,
            'from_type': 'trip',
            'to_type': 'depot',
            'from_location': trip['end_location'],
            'to_location': depot_id,
            'distance_miles': distance,
            'travel_time_minutes': travel_time,
            'deadheading_cost': deadheading_cost,
            'waiting_cost': 0,
            'delay_risk_cost': 0,
            'total_cost': deadheading_cost
        }
        connections.append(connection)
    
    # Add trip-to-trip connections
    for i, trip_i in trips_df.iterrows():
        for j, trip_j in trips_df.iterrows():
            # Skip same trip
            if i == j:
                continue
                
            # Check if trip_j can follow trip_i (time compatibility)
            if pd.isnull(trip_i['scheduled_end_time']) or pd.isnull(trip_j['scheduled_start_time']):
                continue
            
            # Get distance between end of trip_i and start of trip_j
            from_loc = trip_i['end_location']
            to_loc = trip_j['start_location']
            
            if from_loc == 'elmst' and to_loc == 'welst':
                distance = DISTANCES['elmst_to_welst']
            elif from_loc == 'welst' and to_loc == 'elmst':
                distance = DISTANCES['welst_to_elmst']
            elif from_loc == to_loc:
                distance = 0.1  # Short distance if same location
            else:
                # Use average for any other combinations
                distance = (DISTANCES['elmst_to_welst'] + DISTANCES['welst_to_elmst']) / 2
            
            # Calculate travel time
            travel_time = calculate_travel_time(distance)
            
            # Calculate earliest possible arrival time at start of trip_j
            earliest_arrival = trip_i['scheduled_end_time'] + timedelta(minutes=travel_time)
            
            # Check if trip_j can be served after trip_i
            if earliest_arrival <= trip_j['scheduled_start_time']:
                # Calculate waiting time
                waiting_time = (trip_j['scheduled_start_time'] - earliest_arrival).total_seconds() / 60
                
                # Calculate delay risk (if trip_i has history of delays)
                if not pd.isnull(trip_i['actual_end_time']) and not pd.isnull(trip_i['scheduled_end_time']):
                    delay = (trip_i['actual_end_time'] - trip_i['scheduled_end_time']).total_seconds() / 60
                    delay_risk = max(0, delay)
                else:
                    delay_risk = 0
                
                # Calculate costs
                deadheading_cost = distance * cost_per_mile + travel_time * cost_per_minute
                waiting_cost = waiting_time * waiting_cost_factor * cost_per_minute
                delay_risk_cost = delay_risk * delay_risk_factor * cost_per_minute
                total_cost = deadheading_cost + waiting_cost + delay_risk_cost
                
                connection = {
                    'from_id': trip_i['half_trip_id'],
                    'to_id': trip_j['half_trip_id'],
                    'from_type': 'trip',
                    'to_type': 'trip',
                    'from_location': trip_i['end_location'],
                    'to_location': trip_j['start_location'],
                    'distance_miles': distance,
                    'travel_time_minutes': travel_time,
                    'waiting_time_minutes': waiting_time,
                    'deadheading_cost': deadheading_cost,
                    'waiting_cost': waiting_cost,
                    'delay_risk_cost': delay_risk_cost,
                    'total_cost': total_cost
                }
                connections.append(connection)
    
    # Convert to DataFrame
    connections_df = pd.DataFrame(connections)
    return connections_df

def solve_arc_flow_mdvsp(trips_df, connections_df, depot_id=DEPOT_ID, max_vehicles=10):
    """
    Solve the MDVSP using the arc-flow formulation
    
    Args:
        trips_df: DataFrame of trip nodes
        connections_df: DataFrame of connections between trips
        depot_id: ID of the depot
        max_vehicles: Maximum number of vehicles available
        
    Returns:
        Solution DataFrame with vehicle schedules
    """
    print("Solving MDVSP using arc-flow formulation...")
    start_time = time.time()
    
    # Create a list of all trips
    trips = trips_df['half_trip_id'].tolist()
    
    # Create a dictionary to store connections
    connections = {}
    for _, conn in connections_df.iterrows():
        from_id = conn['from_id']
        to_id = conn['to_id']
        total_cost = conn['total_cost']
        
        # Add connection to the dictionary
        if from_id not in connections:
            connections[from_id] = {}
        connections[from_id][to_id] = total_cost
    
    # Create the PuLP model
    model = pl.LpProblem("MDVSP_ArcFlow", pl.LpMinimize)
    
    # Create variables
    x = {}
    for _, conn in connections_df.iterrows():
        from_id = conn['from_id']
        to_id = conn['to_id']
        
        # Create binary variable for this connection
        x[(from_id, to_id)] = pl.LpVariable(f"x_{from_id}_{to_id}", cat='Binary')
    
    # Objective function: minimize total cost
    model += pl.lpSum([x[(conn['from_id'], conn['to_id'])] * conn['total_cost'] for _, conn in connections_df.iterrows()])
    
    # Constraint 1: Flow conservation for trips
    for trip in trips:
        # Inflow equals outflow for each trip
        inflow = pl.lpSum([x[(from_id, trip)] for from_id in connections if trip in connections[from_id]])
        outflow = pl.lpSum([x[(trip, to_id)] for to_id in connections.get(trip, {})])
        
        model += inflow == 1, f"Trip_{trip}_must_be_served"
        model += inflow == outflow, f"Flow_conservation_{trip}"
    
    # Constraint 2: Maximum number of vehicles
    model += pl.lpSum([x[(depot_id, to_id)] for to_id in connections.get(depot_id, {})]) <= max_vehicles, "Max_vehicles"
    
    # Solve the model
    solver = pl.PULP_CBC_CMD(msg=True, timeLimit=300)
    model.solve(solver)
    
    end_time = time.time()
    solution_time = end_time - start_time
    
    print(f"Status: {pl.LpStatus[model.status]}")
    print(f"Total cost: {pl.value(model.objective)}")
    print(f"Solution time: {solution_time:.2f} seconds")
    
    # Extract solution
    if model.status == pl.LpStatusOptimal:
        # Create a list to store vehicle schedules
        vehicle_schedules = []
        
        # Start with connections from depot
        for to_id in connections.get(depot_id, {}):
            if pl.value(x[(depot_id, to_id)]) > 0.5:  # If this connection is used
                # Start a new vehicle schedule
                schedule = [{'type': 'depot', 'id': depot_id, 'location': depot_id}]
                
                # Follow the path of this vehicle
                current_id = to_id
                while current_id != depot_id:
                    # Add this trip to the schedule
                    trip_info = trips_df[trips_df['half_trip_id'] == current_id].iloc[0].to_dict()
                    schedule.append({
                        'type': 'trip',
                        'id': current_id,
                        'start_location': trip_info['start_location'],
                        'end_location': trip_info['end_location'],
                        'start_time': trip_info['scheduled_start_time'],
                        'end_time': trip_info['scheduled_end_time'],
                        'route_id': trip_info['route_id'],
                        'direction_id': trip_info['direction_id']
                    })
                    
                    # Find the next trip/depot
                    next_id = None
                    for possible_next in connections.get(current_id, {}):
                        if pl.value(x[(current_id, possible_next)]) > 0.5:
                            next_id = possible_next
                            break
                    
                    if next_id is None:
                        print(f"Warning: No next trip found for {current_id}")
                        break
                        
                    current_id = next_id
                
                # Close the schedule with return to depot
                schedule.append({'type': 'depot', 'id': depot_id, 'location': depot_id})
                
                # Add this schedule to the list
                vehicle_schedules.append(schedule)
        
        # Convert to DataFrame for easier analysis
        schedule_rows = []
        for vehicle_id, schedule in enumerate(vehicle_schedules):
            for stop_num, stop in enumerate(schedule):
                row = {
                    'vehicle_id': vehicle_id + 1,
                    'stop_num': stop_num + 1,
                    'type': stop['type'],
                    'id': stop['id']
                }
                
                # Add location information
                if 'location' in stop:
                    row['location'] = stop['location']
                
                # Add trip details if this is a trip
                if stop['type'] == 'trip':
                    row.update({
                        'start_location': stop['start_location'],
                        'end_location': stop['end_location'],
                        'start_time': stop['start_time'],
                        'end_time': stop['end_time'],
                        'route_id': stop['route_id'],
                        'direction_id': stop['direction_id']
                    })
                
                schedule_rows.append(row)
        
        schedules_df = pd.DataFrame(schedule_rows)
        return schedules_df
    else:
        print("No optimal solution found.")
        return None

def solve_set_partitioning_mdvsp(trips_df, connections_df, depot_id='fellsway_garage', max_vehicles=10):
    """
    Solve the MDVSP using the set partitioning formulation
    
    Args:
        trips_df: DataFrame of trip nodes
        connections_df: DataFrame of connections between trips
        depot_id: ID of the depot
        max_vehicles: Maximum number of vehicles available
        
    Returns:
        Solution DataFrame with vehicle schedules
    """
    print("Solving MDVSP using set partitioning formulation...")
    start_time = time.time()
    
    # Create a list of all trips
    trips = trips_df['half_trip_id'].tolist()
    
    # Create a graph representation for the network
    graph = {}
    for _, conn in connections_df.iterrows():
        from_id = conn['from_id']
        to_id = conn['to_id']
        cost = conn['total_cost']
        
        if from_id not in graph:
            graph[from_id] = {}
        graph[from_id][to_id] = cost
    
    # Generate all feasible vehicle schedules (paths from depot to depot)
    print("Generating feasible vehicle schedules...")
    feasible_schedules = []
    
    # Use a recursive function to find all paths from depot to depot
    def find_paths(current_path, visited_trips, current_cost):
        current_node = current_path[-1]
        
        # If we've returned to the depot and visited at least one trip, we have a feasible schedule
        if current_node == depot_id and len(current_path) > 2:
            # Extract just the trip IDs (exclude depot at start and end)
            trip_ids = [node for node in current_path[1:-1]]
            feasible_schedules.append({
                'trips': trip_ids,
                'cost': current_cost,
                'path': current_path.copy()  # Keep full path for reference
            })
            return
        
        # If we're at the maximum path length, stop recursion
        if len(current_path) > len(trips) + 2:
            return
        
        # Try all possible next nodes
        if current_node in graph:
            for next_node, edge_cost in graph[current_node].items():
                # Skip if this trip has already been visited
                if next_node in trips and next_node in visited_trips:
                    continue
                
                # Add this node to the path
                new_path = current_path + [next_node]
                new_visited = visited_trips.copy()
                if next_node in trips:
                    new_visited.add(next_node)
                
                # Continue recursion
                find_paths(new_path, new_visited, current_cost + edge_cost)
    
    # Start with paths from the depot
    find_paths([depot_id], set(), 0)
    
    print(f"Generated {len(feasible_schedules)} feasible schedules")
    
    # Create the PuLP model
    model = pl.LpProblem("MDVSP_SetPartitioning", pl.LpMinimize)
    
    # Create variables - one for each feasible schedule
    y = {}
    for i, schedule in enumerate(feasible_schedules):
        y[i] = pl.LpVariable(f"y_{i}", cat='Binary')
    
    # Objective function: minimize total cost
    model += pl.lpSum([y[i] * schedule['cost'] for i, schedule in enumerate(feasible_schedules)])
    
    # Constraint 1: Each trip must be covered exactly once
    for trip in trips:
        model += pl.lpSum([y[i] for i, schedule in enumerate(feasible_schedules) if trip in schedule['trips']]) == 1, f"Trip_{trip}_must_be_served"
    
    # Constraint 2: Maximum number of vehicles
    model += pl.lpSum([y[i] for i in range(len(feasible_schedules))]) <= max_vehicles, "Max_vehicles"
    
    # Solve the model
    solver = pl.PULP_CBC_CMD(msg=True, timeLimit=300)
    model.solve(solver)
    
    end_time = time.time()
    solution_time = end_time - start_time
    
    print(f"Status: {pl.LpStatus[model.status]}")
    print(f"Total cost: {pl.value(model.objective)}")
    print(f"Solution time: {solution_time:.2f} seconds")
    
    # Extract solution
    if model.status == pl.LpStatusOptimal:
        # Create a list to store vehicle schedules
        schedule_rows = []
        
        # Process each selected schedule
        vehicle_id = 1
        for i, schedule in enumerate(feasible_schedules):
            if pl.value(y[i]) > 0.5:  # If this schedule is used
                path = schedule['path']
                
                # Add depot departure
                schedule_rows.append({
                    'vehicle_id': vehicle_id,
                    'stop_num': 1,
                    'type': 'depot',
                    'id': depot_id,
                    'location': depot_id
                })
                
                # Add each trip in the schedule
                for stop_num, trip_id in enumerate(schedule['trips']):
                    # Get trip details
                    trip_info = trips_df[trips_df['half_trip_id'] == trip_id].iloc[0].to_dict()
                    
                    schedule_rows.append({
                        'vehicle_id': vehicle_id,
                        'stop_num': stop_num + 2,
                        'type': 'trip',
                        'id': trip_id,
                        'start_location': trip_info['start_location'],
                        'end_location': trip_info['end_location'],
                        'start_time': trip_info['scheduled_start_time'],
                        'end_time': trip_info['scheduled_end_time'],
                        'route_id': trip_info['route_id'],
                        'direction_id': trip_info['direction_id']
                    })
                
                # Add depot return
                schedule_rows.append({
                    'vehicle_id': vehicle_id,
                    'stop_num': len(schedule['trips']) + 2,
                    'type': 'depot',
                    'id': depot_id,
                    'location': depot_id
                })
                
                vehicle_id += 1
        
        schedules_df = pd.DataFrame(schedule_rows)
        return schedules_df
    else:
        print("No optimal solution found.")
        return None

def analyze_lp_relaxation_set_partitioning(trips_df, connections_df, depot_id='fellsway_garage', max_vehicles=10):
    """
    Analyze the LP relaxation of set partitioning formulation
    
    Args:
        trips_df: DataFrame of trip nodes
        connections_df: DataFrame of connections between trips
        depot_id: ID of the depot
        max_vehicles: Maximum number of vehicles available
    """
    print("Analyzing LP relaxation of set partitioning formulation...")
    start_time = time.time()
    
    # Create a list of all trips
    trips = trips_df['half_trip_id'].tolist()
    
    # Create a graph representation for the network
    graph = {}
    for _, conn in connections_df.iterrows():
        from_id = conn['from_id']
        to_id = conn['to_id']
        cost = conn['total_cost']
        
        if from_id not in graph:
            graph[from_id] = {}
        graph[from_id][to_id] = cost
    
    # Generate all feasible vehicle schedules
    feasible_schedules = []
    
    # Use a recursive function to find all paths from depot to depot
    def find_paths(current_path, visited_trips, current_cost):
        current_node = current_path[-1]
        
        # If we've returned to the depot and visited at least one trip, we have a feasible schedule
        if current_node == depot_id and len(current_path) > 2:
            # Extract just the trip IDs (exclude depot at start and end)
            trip_ids = [node for node in current_path[1:-1]]
            feasible_schedules.append({
                'trips': trip_ids,
                'cost': current_cost
            })
            return
        
        # If we're at the maximum path length, stop recursion
        if len(current_path) > len(trips) + 2:
            return
        
        # Try all possible next nodes
        if current_node in graph:
            for next_node, edge_cost in graph[current_node].items():
                # Skip if this trip has already been visited
                if next_node in trips and next_node in visited_trips:
                    continue
                
                # Add this node to the path
                new_path = current_path + [next_node]
                new_visited = visited_trips.copy()
                if next_node in trips:
                    new_visited.add(next_node)
                
                # Continue recursion
                find_paths(new_path, new_visited, current_cost + edge_cost)
    
    # Start with paths from the depot
    find_paths([depot_id], set(), 0)
    
    print(f"Generated {len(feasible_schedules)} feasible schedules")
    
    # Create the PuLP model
    model = pl.LpProblem("MDVSP_SetPartitioning_LP", pl.LpMinimize)
    
    # Create continuous variables (for LP relaxation)
    y = {}
    for i, schedule in enumerate(feasible_schedules):
        y[i] = pl.LpVariable(f"y_{i}", lowBound=0, upBound=1, cat='Continuous')
    
    # Objective function: minimize total cost
    model += pl.lpSum([y[i] * schedule['cost'] for i, schedule in enumerate(feasible_schedules)])
    
    # Constraint 1: Each trip must be covered exactly once
    for trip in trips:
        model += pl.lpSum([y[i] for i, schedule in enumerate(feasible_schedules) if trip in schedule['trips']]) == 1, f"Trip_{trip}_must_be_served"
    
    # Constraint 2: Maximum number of vehicles
    model += pl.lpSum([y[i] for i in range(len(feasible_schedules))]) <= max_vehicles, "Max_vehicles"
    
    # Solve the LP relaxation
    solver = pl.PULP_CBC_CMD(msg=False)
    model.solve(solver)
    
    end_time = time.time()
    solution_time = end_time - start_time
    
    print(f"Set partitioning LP relaxation - Status: {pl.LpStatus[model.status]}")
    print(f"Set partitioning LP relaxation - Objective: {pl.value(model.objective)}")
    print(f"Solution time: {solution_time:.2f} seconds")

def analyze_lp_relaxation(trips_df, connections_df, depot_id=DEPOT_ID, max_vehicles=10):
    """
    Analyze the LP relaxation of arc-flow formulation
    
    Args:
        trips_df: DataFrame of trip nodes
        connections_df: DataFrame of connections between trips
        depot_id: ID of the depot
        max_vehicles: Maximum number of vehicles available
    """
    print("Analyzing LP relaxation of arc-flow formulation...")
    
    # Create a list of all trips
    trips = trips_df['half_trip_id'].tolist()
    
    # Create a dictionary to store connections
    connections = {}
    for _, conn in connections_df.iterrows():
        from_id = conn['from_id']
        to_id = conn['to_id']
        total_cost = conn['total_cost']
        
        # Add connection to the dictionary
        if from_id not in connections:
            connections[from_id] = {}
        connections[from_id][to_id] = total_cost
    
    # Create the PuLP model for arc-flow
    arc_flow_model = pl.LpProblem("MDVSP_ArcFlow_LP", pl.LpMinimize)
    
    # Create continuous variables
    x = {}
    for _, conn in connections_df.iterrows():
        from_id = conn['from_id']
        to_id = conn['to_id']
        
        # Create continuous variable for this connection
        x[(from_id, to_id)] = pl.LpVariable(f"x_{from_id}_{to_id}", lowBound=0, upBound=1, cat='Continuous')
    
    # Objective function: minimize total cost
    arc_flow_model += pl.lpSum([x[(conn['from_id'], conn['to_id'])] * conn['total_cost'] for _, conn in connections_df.iterrows()])
    
    # Constraint 1: Flow conservation for trips
    for trip in trips:
        # Inflow equals outflow for each trip
        inflow = pl.lpSum([x[(from_id, trip)] for from_id in connections if trip in connections[from_id]])
        outflow = pl.lpSum([x[(trip, to_id)] for to_id in connections.get(trip, {})])
        
        arc_flow_model += inflow == 1, f"Trip_{trip}_must_be_served"
        arc_flow_model += inflow == outflow, f"Flow_conservation_{trip}"
    
    # Constraint 2: Maximum number of vehicles
    arc_flow_model += pl.lpSum([x[(depot_id, to_id)] for to_id in connections.get(depot_id, {})]) <= max_vehicles, "Max_vehicles"
    
    # Solve the LP relaxation
    solver = pl.PULP_CBC_CMD(msg=False)
    arc_flow_model.solve(solver)
    
    print(f"Arc-flow LP relaxation - Status: {pl.LpStatus[arc_flow_model.status]}")
    print(f"Arc-flow LP relaxation - Objective: {pl.value(arc_flow_model.objective)}")

def main():
    # Input and output paths
    input_csv = 'filtered_route100_4h_7h.csv'
    trip_nodes_output = 'route100_4h_7h_trip_nodes_pulp.csv'
    connections_output = 'route100_4h_7h_connections_pulp.csv'
    arc_flow_solution_output = 'route100_4h_7h_arc_flow_solution_pulp.csv'
    set_partitioning_solution_output = 'route100_4h_7h_set_partitioning_solution_pulp.csv'
    
    try:
        # Step 1: Create trip nodes
        print("\n--- STEP 1: Creating trip nodes ---")
        trips_df = create_trip_nodes(input_csv)
        
        # Print summary
        print(f"\nCreated {len(trips_df)} trip nodes")
        if not trips_df.empty:
            print("\nSample trip nodes:")
            print(trips_df.head())
            
            # Save to CSV
            trips_df.to_csv(trip_nodes_output, index=False)
            print(f"\nTrip nodes saved to {trip_nodes_output}")
        
        # Step 2: Calculate connections and costs
        print("\n--- STEP 2: Calculating connections and costs ---")
        connections_df = calculate_costs(trips_df)
        
        # Print summary
        print(f"\nCreated {len(connections_df)} connections")
        print("\nSample connections:")
        print(connections_df.head())
        
        # Save to CSV
        connections_df.to_csv(connections_output, index=False)
        print(f"\nConnections saved to {connections_output}")
        
        # Step 3: Analyze LP relaxation of arc-flow formulation
        print("\n--- STEP 3: Analyzing LP relaxation of arc-flow formulation ---")
        analyze_lp_relaxation(trips_df, connections_df)
        
        # Step 4: Analyze LP relaxation of set partitioning formulation
        print("\n--- STEP 4: Analyzing LP relaxation of set partitioning formulation ---")
        analyze_lp_relaxation_set_partitioning(trips_df, connections_df)
        
        # Step 5: Solve using arc-flow formulation
        print("\n--- STEP 5: Solving with arc-flow formulation ---")
        arc_flow_solution = solve_arc_flow_mdvsp(trips_df, connections_df)
        
        if arc_flow_solution is not None:
            print("\nSample arc-flow solution:")
            print(arc_flow_solution.head())
            
            # Save to CSV
            arc_flow_solution.to_csv(arc_flow_solution_output, index=False)
            print(f"\nArc-flow solution saved to {arc_flow_solution_output}")
        
        # Step 6: Solve using set partitioning formulation
        print("\n--- STEP 6: Solving with set partitioning formulation ---")
        set_partitioning_solution = solve_set_partitioning_mdvsp(trips_df, connections_df)
        
        if set_partitioning_solution is not None:
            print("\nSample set partitioning solution:")
            print(set_partitioning_solution.head())
            
            # Save to CSV
            set_partitioning_solution.to_csv(set_partitioning_solution_output, index=False)
            print(f"\nSet partitioning solution saved to {set_partitioning_solution_output}")
    
    except Exception as e:
        print(f"Error: {e}")

if __name__ == "__main__":
    main()


--- STEP 1: Creating trip nodes ---
Total records: 25
Unique half_trip_ids: 13
Unique stop_ids: [52711  8301  8302 52720]
Unique time_point_ids: ['welst' 'elm']

Created 12 trip nodes

Sample trip nodes:
   half_trip_id  route_id direction_id start_location end_location  \
0      65802629       100     Outbound          welst          elm   
1      65802630       100      Inbound            elm        welst   
2      65802631       100     Outbound          welst          elm   
3      65802632       100      Inbound            elm        welst   
4      65802633       100     Outbound          welst          elm   

  scheduled_start_time  scheduled_end_time   actual_start_time  \
0  2025-04-29 16:00:00 2025-04-29 16:11:00 2025-04-29 16:06:48   
1  2025-04-29 16:15:00 2025-04-29 16:27:00 2025-04-29 16:23:13   
2  2025-04-29 16:30:00 2025-04-29 16:41:00 2025-04-29 16:38:27   
3  2025-04-29 16:45:00 2025-04-29 16:57:00 2025-04-29 16:52:23   
4  2025-04-29 17:00:00 2025-04-29 17:11:00 2

/var/folders/v8/1793d44577q9lqgx_01s8skm0000gn/T/ipykernel_21153/4054497762.py:87: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  trips_df[col] = pd.to_datetime(trips_df[col], errors='coerce')
/var/folders/v8/1793d44577q9lqgx_01s8skm0000gn/T/ipykernel_21153/4054497762.py:87: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  trips_df[col] = pd.to_datetime(trips_df[col], errors='coerce')
/var/folders/v8/1793d44577q9lqgx_01s8skm0000gn/T/ipykernel_21153/4054497762.py:87: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  trips_df[col] = pd.to_datetime(trips_df[col], errors='coerce')
/var

Arc-flow LP relaxation - Status: Optimal
Arc-flow LP relaxation - Objective: 232.63333333333335

--- STEP 4: Analyzing LP relaxation of set partitioning formulation ---
Analyzing LP relaxation of set partitioning formulation...
Generated 4095 feasible schedules
Set partitioning LP relaxation - Status: Optimal
Set partitioning LP relaxation - Objective: 232.63333333333333
Solution time: 0.09 seconds

--- STEP 5: Solving with arc-flow formulation ---
Solving MDVSP using arc-flow formulation...
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/helmadevina/Desktop/sklearn-env/lib/python3.12/site-packages/pulp/apis/../solverdir/cbc/osx/i64/cbc /var/folders/v8/1793d44577q9lqgx_01s8skm0000gn/T/e6768e3b14f4451cb96ab10e334d85d2-pulp.mps -sec 300 -timeMode elapsed -branch -printingOptions all -solution /var/folders/v8/1793d44577q9lqgx_01s8skm0000gn/T/e6768e3b14f4451cb96ab10e334d85d2-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line

## b. 4 hours

In [4]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import pulp as pl
import time

# Define bus operation parameters
BUS_PARAMS = {
    'avg_speed_mph': 12,  # Average bus speed in mph
    'cost_per_mile': 2.5,  # Operating cost per mile
    'cost_per_minute': 1.0,  # Cost per minute of operation
    'waiting_cost_factor': 0.5,  # Cost factor for waiting time
    'delay_risk_factor': 2.0  # Cost factor for delay risk
}

# Define the depot
DEPOT_ID = 'fellsway_garage'

# Define distances from Google Maps screenshots (in miles)
DISTANCES = {
    'depot_to_welst': 2.0,  # Fellsway Garage to Wellington St
    'depot_to_elmst': 1.4,  # Fellsway Garage to Elm St
    'welst_to_depot': 2.0,  # Wellington St to Fellsway Garage
    'elmst_to_depot': 1.4,  # Elm St to Fellsway Garage
    'elmst_to_welst': 1.5,  # Estimated distance between stops
    'welst_to_elmst': 1.5   # Estimated distance between stops
}

def calculate_travel_time(distance, speed=BUS_PARAMS['avg_speed_mph']):
    """Calculate travel time based on distance and speed"""
    return (distance / speed) * 60

def create_trip_nodes(csv_path):
    """Create trip nodes from the filtered CSV data"""
    # Read the filtered data
    df = pd.read_csv(csv_path)
    
    # Print data summary
    print(f"Total records: {len(df)}")
    print(f"Unique half_trip_ids: {df['half_trip_id'].nunique()}")
    print(f"Unique stop_ids: {df['stop_id'].unique()}")
    print(f"Unique time_point_ids: {df['time_point_id'].unique()}")
    
    # Map time_point_ids to our location codes
    location_map = {
        'elmst': 'elmst',
        'welst': 'welst'
    }
    
    # Group by half_trip_id
    trips = []
    
    # For each half_trip_id, find its startpoint and endpoint
    for half_trip_id, group in df.groupby('half_trip_id'):
        startpoints = group[group['point_type'] == 'Startpoint']
        endpoints = group[group['point_type'] == 'Endpoint']
        
        if not startpoints.empty and not endpoints.empty:
            # Get start and end locations
            start_time_point = startpoints['time_point_id'].iloc[0]
            end_time_point = endpoints['time_point_id'].iloc[0]
            
            # Map to our location codes
            start_location = location_map.get(start_time_point, start_time_point)
            end_location = location_map.get(end_time_point, end_time_point)
            
            trip = {
                'half_trip_id': half_trip_id,
                'route_id': group['route_id'].iloc[0],
                'direction_id': group['direction_id'].iloc[0],
                'start_location': start_location,
                'end_location': end_location,
                'scheduled_start_time': startpoints['scheduled'].iloc[0],
                'scheduled_end_time': endpoints['scheduled'].iloc[0],
                'actual_start_time': startpoints['actual'].iloc[0],
                'actual_end_time': endpoints['actual'].iloc[0]
            }
            trips.append(trip)
    
    # Convert to DataFrame
    trips_df = pd.DataFrame(trips)
    
    # Parse datetime strings
    for col in ['scheduled_start_time', 'scheduled_end_time', 'actual_start_time', 'actual_end_time']:
        if col in trips_df.columns:
            # Convert times to datetime objects
            trips_df[col] = pd.to_datetime(trips_df[col], errors='coerce')
    
    return trips_df

def calculate_costs(trips_df, depot_id=DEPOT_ID):
    """
    Calculate costs between trips and depot connections using distances from Google Maps
    
    Args:
        trips_df: DataFrame of trip nodes
        depot_id: ID of the depot
        
    Returns:
        DataFrame with trip-to-trip costs and depot connections
    """
    # Get parameters
    cost_per_mile = BUS_PARAMS['cost_per_mile']
    cost_per_minute = BUS_PARAMS['cost_per_minute']
    waiting_cost_factor = BUS_PARAMS['waiting_cost_factor']
    delay_risk_factor = BUS_PARAMS['delay_risk_factor']
    
    # Create empty lists to store connections
    connections = []
    
    # Add depot to start connections
    for _, trip in trips_df.iterrows():
        # Get distance from depot to this trip's start location
        if trip['start_location'] == 'welst':
            distance = DISTANCES['depot_to_welst']
        elif trip['start_location'] == 'elmst':
            distance = DISTANCES['depot_to_elmst']
        else:
            # Use average distance for any other locations
            distance = (DISTANCES['depot_to_welst'] + DISTANCES['depot_to_elmst']) / 2
            
        # Calculate travel time
        travel_time = calculate_travel_time(distance)
        
        # Calculate costs
        deadheading_cost = distance * cost_per_mile + travel_time * cost_per_minute
        
        connection = {
            'from_id': depot_id,
            'to_id': trip['half_trip_id'],
            'from_type': 'depot',
            'to_type': 'trip',
            'from_location': depot_id,
            'to_location': trip['start_location'],
            'distance_miles': distance,
            'travel_time_minutes': travel_time,
            'deadheading_cost': deadheading_cost,
            'waiting_cost': 0,
            'delay_risk_cost': 0,
            'total_cost': deadheading_cost
        }
        connections.append(connection)
    
    # Add end to depot connections
    for _, trip in trips_df.iterrows():
        # Get distance from this trip's end location to depot
        if trip['end_location'] == 'welst':
            distance = DISTANCES['welst_to_depot']
        elif trip['end_location'] == 'elmst':
            distance = DISTANCES['elmst_to_depot']
        else:
            # Use average distance for any other locations
            distance = (DISTANCES['welst_to_depot'] + DISTANCES['elmst_to_depot']) / 2
            
        # Calculate travel time
        travel_time = calculate_travel_time(distance)
        
        # Calculate costs
        deadheading_cost = distance * cost_per_mile + travel_time * cost_per_minute
        
        connection = {
            'from_id': trip['half_trip_id'],
            'to_id': depot_id,
            'from_type': 'trip',
            'to_type': 'depot',
            'from_location': trip['end_location'],
            'to_location': depot_id,
            'distance_miles': distance,
            'travel_time_minutes': travel_time,
            'deadheading_cost': deadheading_cost,
            'waiting_cost': 0,
            'delay_risk_cost': 0,
            'total_cost': deadheading_cost
        }
        connections.append(connection)
    
    # Add trip-to-trip connections
    for i, trip_i in trips_df.iterrows():
        for j, trip_j in trips_df.iterrows():
            # Skip same trip
            if i == j:
                continue
                
            # Check if trip_j can follow trip_i (time compatibility)
            if pd.isnull(trip_i['scheduled_end_time']) or pd.isnull(trip_j['scheduled_start_time']):
                continue
            
            # Get distance between end of trip_i and start of trip_j
            from_loc = trip_i['end_location']
            to_loc = trip_j['start_location']
            
            if from_loc == 'elmst' and to_loc == 'welst':
                distance = DISTANCES['elmst_to_welst']
            elif from_loc == 'welst' and to_loc == 'elmst':
                distance = DISTANCES['welst_to_elmst']
            elif from_loc == to_loc:
                distance = 0.1  # Short distance if same location
            else:
                # Use average for any other combinations
                distance = (DISTANCES['elmst_to_welst'] + DISTANCES['welst_to_elmst']) / 2
            
            # Calculate travel time
            travel_time = calculate_travel_time(distance)
            
            # Calculate earliest possible arrival time at start of trip_j
            earliest_arrival = trip_i['scheduled_end_time'] + timedelta(minutes=travel_time)
            
            # Check if trip_j can be served after trip_i
            if earliest_arrival <= trip_j['scheduled_start_time']:
                # Calculate waiting time
                waiting_time = (trip_j['scheduled_start_time'] - earliest_arrival).total_seconds() / 60
                
                # Calculate delay risk (if trip_i has history of delays)
                if not pd.isnull(trip_i['actual_end_time']) and not pd.isnull(trip_i['scheduled_end_time']):
                    delay = (trip_i['actual_end_time'] - trip_i['scheduled_end_time']).total_seconds() / 60
                    delay_risk = max(0, delay)
                else:
                    delay_risk = 0
                
                # Calculate costs
                deadheading_cost = distance * cost_per_mile + travel_time * cost_per_minute
                waiting_cost = waiting_time * waiting_cost_factor * cost_per_minute
                delay_risk_cost = delay_risk * delay_risk_factor * cost_per_minute
                total_cost = deadheading_cost + waiting_cost + delay_risk_cost
                
                connection = {
                    'from_id': trip_i['half_trip_id'],
                    'to_id': trip_j['half_trip_id'],
                    'from_type': 'trip',
                    'to_type': 'trip',
                    'from_location': trip_i['end_location'],
                    'to_location': trip_j['start_location'],
                    'distance_miles': distance,
                    'travel_time_minutes': travel_time,
                    'waiting_time_minutes': waiting_time,
                    'deadheading_cost': deadheading_cost,
                    'waiting_cost': waiting_cost,
                    'delay_risk_cost': delay_risk_cost,
                    'total_cost': total_cost
                }
                connections.append(connection)
    
    # Convert to DataFrame
    connections_df = pd.DataFrame(connections)
    return connections_df

def solve_arc_flow_mdvsp(trips_df, connections_df, depot_id=DEPOT_ID, max_vehicles=10):
    """
    Solve the MDVSP using the arc-flow formulation
    
    Args:
        trips_df: DataFrame of trip nodes
        connections_df: DataFrame of connections between trips
        depot_id: ID of the depot
        max_vehicles: Maximum number of vehicles available
        
    Returns:
        Solution DataFrame with vehicle schedules
    """
    print("Solving MDVSP using arc-flow formulation...")
    start_time = time.time()
    
    # Create a list of all trips
    trips = trips_df['half_trip_id'].tolist()
    
    # Create a dictionary to store connections
    connections = {}
    for _, conn in connections_df.iterrows():
        from_id = conn['from_id']
        to_id = conn['to_id']
        total_cost = conn['total_cost']
        
        # Add connection to the dictionary
        if from_id not in connections:
            connections[from_id] = {}
        connections[from_id][to_id] = total_cost
    
    # Create the PuLP model
    model = pl.LpProblem("MDVSP_ArcFlow", pl.LpMinimize)
    
    # Create variables
    x = {}
    for _, conn in connections_df.iterrows():
        from_id = conn['from_id']
        to_id = conn['to_id']
        
        # Create binary variable for this connection
        x[(from_id, to_id)] = pl.LpVariable(f"x_{from_id}_{to_id}", cat='Binary')
    
    # Objective function: minimize total cost
    model += pl.lpSum([x[(conn['from_id'], conn['to_id'])] * conn['total_cost'] for _, conn in connections_df.iterrows()])
    
    # Constraint 1: Flow conservation for trips
    for trip in trips:
        # Inflow equals outflow for each trip
        inflow = pl.lpSum([x[(from_id, trip)] for from_id in connections if trip in connections[from_id]])
        outflow = pl.lpSum([x[(trip, to_id)] for to_id in connections.get(trip, {})])
        
        model += inflow == 1, f"Trip_{trip}_must_be_served"
        model += inflow == outflow, f"Flow_conservation_{trip}"
    
    # Constraint 2: Maximum number of vehicles
    model += pl.lpSum([x[(depot_id, to_id)] for to_id in connections.get(depot_id, {})]) <= max_vehicles, "Max_vehicles"
    
    # Solve the model
    solver = pl.PULP_CBC_CMD(msg=True, timeLimit=300)
    model.solve(solver)
    
    end_time = time.time()
    solution_time = end_time - start_time
    
    print(f"Status: {pl.LpStatus[model.status]}")
    print(f"Total cost: {pl.value(model.objective)}")
    print(f"Solution time: {solution_time:.2f} seconds")
    
    # Extract solution
    if model.status == pl.LpStatusOptimal:
        # Create a list to store vehicle schedules
        vehicle_schedules = []
        
        # Start with connections from depot
        for to_id in connections.get(depot_id, {}):
            if pl.value(x[(depot_id, to_id)]) > 0.5:  # If this connection is used
                # Start a new vehicle schedule
                schedule = [{'type': 'depot', 'id': depot_id, 'location': depot_id}]
                
                # Follow the path of this vehicle
                current_id = to_id
                while current_id != depot_id:
                    # Add this trip to the schedule
                    trip_info = trips_df[trips_df['half_trip_id'] == current_id].iloc[0].to_dict()
                    schedule.append({
                        'type': 'trip',
                        'id': current_id,
                        'start_location': trip_info['start_location'],
                        'end_location': trip_info['end_location'],
                        'start_time': trip_info['scheduled_start_time'],
                        'end_time': trip_info['scheduled_end_time'],
                        'route_id': trip_info['route_id'],
                        'direction_id': trip_info['direction_id']
                    })
                    
                    # Find the next trip/depot
                    next_id = None
                    for possible_next in connections.get(current_id, {}):
                        if pl.value(x[(current_id, possible_next)]) > 0.5:
                            next_id = possible_next
                            break
                    
                    if next_id is None:
                        print(f"Warning: No next trip found for {current_id}")
                        break
                        
                    current_id = next_id
                
                # Close the schedule with return to depot
                schedule.append({'type': 'depot', 'id': depot_id, 'location': depot_id})
                
                # Add this schedule to the list
                vehicle_schedules.append(schedule)
        
        # Convert to DataFrame for easier analysis
        schedule_rows = []
        for vehicle_id, schedule in enumerate(vehicle_schedules):
            for stop_num, stop in enumerate(schedule):
                row = {
                    'vehicle_id': vehicle_id + 1,
                    'stop_num': stop_num + 1,
                    'type': stop['type'],
                    'id': stop['id']
                }
                
                # Add location information
                if 'location' in stop:
                    row['location'] = stop['location']
                
                # Add trip details if this is a trip
                if stop['type'] == 'trip':
                    row.update({
                        'start_location': stop['start_location'],
                        'end_location': stop['end_location'],
                        'start_time': stop['start_time'],
                        'end_time': stop['end_time'],
                        'route_id': stop['route_id'],
                        'direction_id': stop['direction_id']
                    })
                
                schedule_rows.append(row)
        
        schedules_df = pd.DataFrame(schedule_rows)
        return schedules_df
    else:
        print("No optimal solution found.")
        return None

def solve_set_partitioning_mdvsp(trips_df, connections_df, depot_id='fellsway_garage', max_vehicles=10):
    """
    Solve the MDVSP using the set partitioning formulation
    
    Args:
        trips_df: DataFrame of trip nodes
        connections_df: DataFrame of connections between trips
        depot_id: ID of the depot
        max_vehicles: Maximum number of vehicles available
        
    Returns:
        Solution DataFrame with vehicle schedules
    """
    print("Solving MDVSP using set partitioning formulation...")
    start_time = time.time()
    
    # Create a list of all trips
    trips = trips_df['half_trip_id'].tolist()
    
    # Create a graph representation for the network
    graph = {}
    for _, conn in connections_df.iterrows():
        from_id = conn['from_id']
        to_id = conn['to_id']
        cost = conn['total_cost']
        
        if from_id not in graph:
            graph[from_id] = {}
        graph[from_id][to_id] = cost
    
    # Generate all feasible vehicle schedules (paths from depot to depot)
    print("Generating feasible vehicle schedules...")
    feasible_schedules = []
    
    # Use a recursive function to find all paths from depot to depot
    def find_paths(current_path, visited_trips, current_cost):
        current_node = current_path[-1]
        
        # If we've returned to the depot and visited at least one trip, we have a feasible schedule
        if current_node == depot_id and len(current_path) > 2:
            # Extract just the trip IDs (exclude depot at start and end)
            trip_ids = [node for node in current_path[1:-1]]
            feasible_schedules.append({
                'trips': trip_ids,
                'cost': current_cost,
                'path': current_path.copy()  # Keep full path for reference
            })
            return
        
        # If we're at the maximum path length, stop recursion
        if len(current_path) > len(trips) + 2:
            return
        
        # Try all possible next nodes
        if current_node in graph:
            for next_node, edge_cost in graph[current_node].items():
                # Skip if this trip has already been visited
                if next_node in trips and next_node in visited_trips:
                    continue
                
                # Add this node to the path
                new_path = current_path + [next_node]
                new_visited = visited_trips.copy()
                if next_node in trips:
                    new_visited.add(next_node)
                
                # Continue recursion
                find_paths(new_path, new_visited, current_cost + edge_cost)
    
    # Start with paths from the depot
    find_paths([depot_id], set(), 0)
    
    print(f"Generated {len(feasible_schedules)} feasible schedules")
    
    # Create the PuLP model
    model = pl.LpProblem("MDVSP_SetPartitioning", pl.LpMinimize)
    
    # Create variables - one for each feasible schedule
    y = {}
    for i, schedule in enumerate(feasible_schedules):
        y[i] = pl.LpVariable(f"y_{i}", cat='Binary')
    
    # Objective function: minimize total cost
    model += pl.lpSum([y[i] * schedule['cost'] for i, schedule in enumerate(feasible_schedules)])
    
    # Constraint 1: Each trip must be covered exactly once
    for trip in trips:
        model += pl.lpSum([y[i] for i, schedule in enumerate(feasible_schedules) if trip in schedule['trips']]) == 1, f"Trip_{trip}_must_be_served"
    
    # Constraint 2: Maximum number of vehicles
    model += pl.lpSum([y[i] for i in range(len(feasible_schedules))]) <= max_vehicles, "Max_vehicles"
    
    # Solve the model
    solver = pl.PULP_CBC_CMD(msg=True, timeLimit=300)
    model.solve(solver)
    
    end_time = time.time()
    solution_time = end_time - start_time
    
    print(f"Status: {pl.LpStatus[model.status]}")
    print(f"Total cost: {pl.value(model.objective)}")
    print(f"Solution time: {solution_time:.2f} seconds")
    
    # Extract solution
    if model.status == pl.LpStatusOptimal:
        # Create a list to store vehicle schedules
        schedule_rows = []
        
        # Process each selected schedule
        vehicle_id = 1
        for i, schedule in enumerate(feasible_schedules):
            if pl.value(y[i]) > 0.5:  # If this schedule is used
                path = schedule['path']
                
                # Add depot departure
                schedule_rows.append({
                    'vehicle_id': vehicle_id,
                    'stop_num': 1,
                    'type': 'depot',
                    'id': depot_id,
                    'location': depot_id
                })
                
                # Add each trip in the schedule
                for stop_num, trip_id in enumerate(schedule['trips']):
                    # Get trip details
                    trip_info = trips_df[trips_df['half_trip_id'] == trip_id].iloc[0].to_dict()
                    
                    schedule_rows.append({
                        'vehicle_id': vehicle_id,
                        'stop_num': stop_num + 2,
                        'type': 'trip',
                        'id': trip_id,
                        'start_location': trip_info['start_location'],
                        'end_location': trip_info['end_location'],
                        'start_time': trip_info['scheduled_start_time'],
                        'end_time': trip_info['scheduled_end_time'],
                        'route_id': trip_info['route_id'],
                        'direction_id': trip_info['direction_id']
                    })
                
                # Add depot return
                schedule_rows.append({
                    'vehicle_id': vehicle_id,
                    'stop_num': len(schedule['trips']) + 2,
                    'type': 'depot',
                    'id': depot_id,
                    'location': depot_id
                })
                
                vehicle_id += 1
        
        schedules_df = pd.DataFrame(schedule_rows)
        return schedules_df
    else:
        print("No optimal solution found.")
        return None

def analyze_lp_relaxation_set_partitioning(trips_df, connections_df, depot_id='fellsway_garage', max_vehicles=10):
    """
    Analyze the LP relaxation of set partitioning formulation
    
    Args:
        trips_df: DataFrame of trip nodes
        connections_df: DataFrame of connections between trips
        depot_id: ID of the depot
        max_vehicles: Maximum number of vehicles available
    """
    print("Analyzing LP relaxation of set partitioning formulation...")
    start_time = time.time()
    
    # Create a list of all trips
    trips = trips_df['half_trip_id'].tolist()
    
    # Create a graph representation for the network
    graph = {}
    for _, conn in connections_df.iterrows():
        from_id = conn['from_id']
        to_id = conn['to_id']
        cost = conn['total_cost']
        
        if from_id not in graph:
            graph[from_id] = {}
        graph[from_id][to_id] = cost
    
    # Generate all feasible vehicle schedules
    feasible_schedules = []
    
    # Use a recursive function to find all paths from depot to depot
    def find_paths(current_path, visited_trips, current_cost):
        current_node = current_path[-1]
        
        # If we've returned to the depot and visited at least one trip, we have a feasible schedule
        if current_node == depot_id and len(current_path) > 2:
            # Extract just the trip IDs (exclude depot at start and end)
            trip_ids = [node for node in current_path[1:-1]]
            feasible_schedules.append({
                'trips': trip_ids,
                'cost': current_cost
            })
            return
        
        # If we're at the maximum path length, stop recursion
        if len(current_path) > len(trips) + 2:
            return
        
        # Try all possible next nodes
        if current_node in graph:
            for next_node, edge_cost in graph[current_node].items():
                # Skip if this trip has already been visited
                if next_node in trips and next_node in visited_trips:
                    continue
                
                # Add this node to the path
                new_path = current_path + [next_node]
                new_visited = visited_trips.copy()
                if next_node in trips:
                    new_visited.add(next_node)
                
                # Continue recursion
                find_paths(new_path, new_visited, current_cost + edge_cost)
    
    # Start with paths from the depot
    find_paths([depot_id], set(), 0)
    
    print(f"Generated {len(feasible_schedules)} feasible schedules")
    
    # Create the PuLP model
    model = pl.LpProblem("MDVSP_SetPartitioning_LP", pl.LpMinimize)
    
    # Create continuous variables (for LP relaxation)
    y = {}
    for i, schedule in enumerate(feasible_schedules):
        y[i] = pl.LpVariable(f"y_{i}", lowBound=0, upBound=1, cat='Continuous')
    
    # Objective function: minimize total cost
    model += pl.lpSum([y[i] * schedule['cost'] for i, schedule in enumerate(feasible_schedules)])
    
    # Constraint 1: Each trip must be covered exactly once
    for trip in trips:
        model += pl.lpSum([y[i] for i, schedule in enumerate(feasible_schedules) if trip in schedule['trips']]) == 1, f"Trip_{trip}_must_be_served"
    
    # Constraint 2: Maximum number of vehicles
    model += pl.lpSum([y[i] for i in range(len(feasible_schedules))]) <= max_vehicles, "Max_vehicles"
    
    # Solve the LP relaxation
    solver = pl.PULP_CBC_CMD(msg=False)
    model.solve(solver)
    
    end_time = time.time()
    solution_time = end_time - start_time
    
    print(f"Set partitioning LP relaxation - Status: {pl.LpStatus[model.status]}")
    print(f"Set partitioning LP relaxation - Objective: {pl.value(model.objective)}")
    print(f"Solution time: {solution_time:.2f} seconds")

def analyze_lp_relaxation(trips_df, connections_df, depot_id=DEPOT_ID, max_vehicles=10):
    """
    Analyze the LP relaxation of arc-flow formulation
    
    Args:
        trips_df: DataFrame of trip nodes
        connections_df: DataFrame of connections between trips
        depot_id: ID of the depot
        max_vehicles: Maximum number of vehicles available
    """
    print("Analyzing LP relaxation of arc-flow formulation...")
    
    # Create a list of all trips
    trips = trips_df['half_trip_id'].tolist()
    
    # Create a dictionary to store connections
    connections = {}
    for _, conn in connections_df.iterrows():
        from_id = conn['from_id']
        to_id = conn['to_id']
        total_cost = conn['total_cost']
        
        # Add connection to the dictionary
        if from_id not in connections:
            connections[from_id] = {}
        connections[from_id][to_id] = total_cost
    
    # Create the PuLP model for arc-flow
    arc_flow_model = pl.LpProblem("MDVSP_ArcFlow_LP", pl.LpMinimize)
    
    # Create continuous variables
    x = {}
    for _, conn in connections_df.iterrows():
        from_id = conn['from_id']
        to_id = conn['to_id']
        
        # Create continuous variable for this connection
        x[(from_id, to_id)] = pl.LpVariable(f"x_{from_id}_{to_id}", lowBound=0, upBound=1, cat='Continuous')
    
    # Objective function: minimize total cost
    arc_flow_model += pl.lpSum([x[(conn['from_id'], conn['to_id'])] * conn['total_cost'] for _, conn in connections_df.iterrows()])
    
    # Constraint 1: Flow conservation for trips
    for trip in trips:
        # Inflow equals outflow for each trip
        inflow = pl.lpSum([x[(from_id, trip)] for from_id in connections if trip in connections[from_id]])
        outflow = pl.lpSum([x[(trip, to_id)] for to_id in connections.get(trip, {})])
        
        arc_flow_model += inflow == 1, f"Trip_{trip}_must_be_served"
        arc_flow_model += inflow == outflow, f"Flow_conservation_{trip}"
    
    # Constraint 2: Maximum number of vehicles
    arc_flow_model += pl.lpSum([x[(depot_id, to_id)] for to_id in connections.get(depot_id, {})]) <= max_vehicles, "Max_vehicles"
    
    # Solve the LP relaxation
    solver = pl.PULP_CBC_CMD(msg=False)
    arc_flow_model.solve(solver)
    
    print(f"Arc-flow LP relaxation - Status: {pl.LpStatus[arc_flow_model.status]}")
    print(f"Arc-flow LP relaxation - Objective: {pl.value(arc_flow_model.objective)}")

def main():
    # Input and output paths
    input_csv = 'filtered_route100_16h_20h.csv'
    trip_nodes_output = 'route100_16h_20h_trip_nodes_pulp.csv'
    connections_output = 'route100_16h_20h_connections_pulp.csv'
    arc_flow_solution_output = 'route100_16h_20h_arc_flow_solution_pulp.csv'
    set_partitioning_solution_output = 'route100_16h_20h_set_partitioning_solution_pulp.csv'
    
    try:
        # Step 1: Create trip nodes
        print("\n--- STEP 1: Creating trip nodes ---")
        trips_df = create_trip_nodes(input_csv)
        
        # Print summary
        print(f"\nCreated {len(trips_df)} trip nodes")
        if not trips_df.empty:
            print("\nSample trip nodes:")
            print(trips_df.head())
            
            # Save to CSV
            trips_df.to_csv(trip_nodes_output, index=False)
            print(f"\nTrip nodes saved to {trip_nodes_output}")
        
        # Step 2: Calculate connections and costs
        print("\n--- STEP 2: Calculating connections and costs ---")
        connections_df = calculate_costs(trips_df)
        
        # Print summary
        print(f"\nCreated {len(connections_df)} connections")
        print("\nSample connections:")
        print(connections_df.head())
        
        # Save to CSV
        connections_df.to_csv(connections_output, index=False)
        print(f"\nConnections saved to {connections_output}")
        
        # Step 3: Analyze LP relaxation of arc-flow formulation
        print("\n--- STEP 3: Analyzing LP relaxation of arc-flow formulation ---")
        analyze_lp_relaxation(trips_df, connections_df)
        
        # Step 4: Analyze LP relaxation of set partitioning formulation
        print("\n--- STEP 4: Analyzing LP relaxation of set partitioning formulation ---")
        analyze_lp_relaxation_set_partitioning(trips_df, connections_df)
        
        # Step 5: Solve using arc-flow formulation
        print("\n--- STEP 5: Solving with arc-flow formulation ---")
        arc_flow_solution = solve_arc_flow_mdvsp(trips_df, connections_df)
        
        if arc_flow_solution is not None:
            print("\nSample arc-flow solution:")
            print(arc_flow_solution.head())
            
            # Save to CSV
            arc_flow_solution.to_csv(arc_flow_solution_output, index=False)
            print(f"\nArc-flow solution saved to {arc_flow_solution_output}")
        
        # Step 6: Solve using set partitioning formulation
        print("\n--- STEP 6: Solving with set partitioning formulation ---")
        set_partitioning_solution = solve_set_partitioning_mdvsp(trips_df, connections_df)
        
        if set_partitioning_solution is not None:
            print("\nSample set partitioning solution:")
            print(set_partitioning_solution.head())
            
            # Save to CSV
            set_partitioning_solution.to_csv(set_partitioning_solution_output, index=False)
            print(f"\nSet partitioning solution saved to {set_partitioning_solution_output}")
    
    except Exception as e:
        print(f"Error: {e}")

if __name__ == "__main__":
    main()


--- STEP 1: Creating trip nodes ---
Total records: 40
Unique half_trip_ids: 20
Unique stop_ids: [52711  8301  8302 52720]
Unique time_point_ids: ['welst' 'elm']

Created 20 trip nodes

Sample trip nodes:
   half_trip_id  route_id direction_id start_location end_location  \
0    65802629.0       100     Outbound          welst          elm   
1    65802630.0       100      Inbound            elm        welst   
2    65802631.0       100     Outbound          welst          elm   
3    65802632.0       100      Inbound            elm        welst   
4    65802633.0       100     Outbound          welst          elm   

  scheduled_start_time  scheduled_end_time   actual_start_time  \
0  2025-04-29 16:00:00 2025-04-29 16:11:00 2025-04-29 16:06:48   
1  2025-04-29 16:15:00 2025-04-29 16:27:00 2025-04-29 16:23:13   
2  2025-04-29 16:30:00 2025-04-29 16:41:00 2025-04-29 16:38:27   
3  2025-04-29 16:45:00 2025-04-29 16:57:00 2025-04-29 16:52:23   
4  2025-04-29 17:00:00 2025-04-29 17:11:00 2

/var/folders/v8/1793d44577q9lqgx_01s8skm0000gn/T/ipykernel_21153/3865941237.py:87: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  trips_df[col] = pd.to_datetime(trips_df[col], errors='coerce')
/var/folders/v8/1793d44577q9lqgx_01s8skm0000gn/T/ipykernel_21153/3865941237.py:87: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  trips_df[col] = pd.to_datetime(trips_df[col], errors='coerce')
/var/folders/v8/1793d44577q9lqgx_01s8skm0000gn/T/ipykernel_21153/3865941237.py:87: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  trips_df[col] = pd.to_datetime(trips_df[col], errors='coerce')
/var

Generated 1048575 feasible schedules
Set partitioning LP relaxation - Status: Optimal
Set partitioning LP relaxation - Objective: 313.16666666666663
Solution time: 47.85 seconds

--- STEP 5: Solving with arc-flow formulation ---
Solving MDVSP using arc-flow formulation...
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/helmadevina/Desktop/sklearn-env/lib/python3.12/site-packages/pulp/apis/../solverdir/cbc/osx/i64/cbc /var/folders/v8/1793d44577q9lqgx_01s8skm0000gn/T/ab5942461017435c9fae5a8b90c06f2f-pulp.mps -sec 300 -timeMode elapsed -branch -printingOptions all -solution /var/folders/v8/1793d44577q9lqgx_01s8skm0000gn/T/ab5942461017435c9fae5a8b90c06f2f-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 46 COLUMNS
At line 1387 RHS
At line 1429 BOUNDS
At line 1660 ENDATA
Problem MODEL has 41 rows, 230 columns and 650 elements
Coin0008I MODEL read with 0 errors
seconds was changed from 1e+100 to 300
Option for

## c. 5 hours

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import pulp as pl
import time

# Define bus operation parameters
BUS_PARAMS = {
    'avg_speed_mph': 12,  # Average bus speed in mph
    'cost_per_mile': 2.5,  # Operating cost per mile
    'cost_per_minute': 1.0,  # Cost per minute of operation
    'waiting_cost_factor': 0.5,  # Cost factor for waiting time
    'delay_risk_factor': 2.0  # Cost factor for delay risk
}

# Define the depot
DEPOT_ID = 'fellsway_garage'

# Define distances from Google Maps screenshots (in miles)
DISTANCES = {
    'depot_to_welst': 2.0,  # Fellsway Garage to Wellington St
    'depot_to_elmst': 1.4,  # Fellsway Garage to Elm St
    'welst_to_depot': 2.0,  # Wellington St to Fellsway Garage
    'elmst_to_depot': 1.4,  # Elm St to Fellsway Garage
    'elmst_to_welst': 1.5,  # Estimated distance between stops
    'welst_to_elmst': 1.5   # Estimated distance between stops
}

def calculate_travel_time(distance, speed=BUS_PARAMS['avg_speed_mph']):
    """Calculate travel time based on distance and speed"""
    return (distance / speed) * 60

def create_trip_nodes(csv_path):
    """Create trip nodes from the filtered CSV data"""
    # Read the filtered data
    df = pd.read_csv(csv_path)
    
    # Print data summary
    print(f"Total records: {len(df)}")
    print(f"Unique half_trip_ids: {df['half_trip_id'].nunique()}")
    print(f"Unique stop_ids: {df['stop_id'].unique()}")
    print(f"Unique time_point_ids: {df['time_point_id'].unique()}")
    
    # Map time_point_ids to our location codes
    location_map = {
        'elmst': 'elmst',
        'welst': 'welst'
    }
    
    # Group by half_trip_id
    trips = []
    
    # For each half_trip_id, find its startpoint and endpoint
    for half_trip_id, group in df.groupby('half_trip_id'):
        startpoints = group[group['point_type'] == 'Startpoint']
        endpoints = group[group['point_type'] == 'Endpoint']
        
        if not startpoints.empty and not endpoints.empty:
            # Get start and end locations
            start_time_point = startpoints['time_point_id'].iloc[0]
            end_time_point = endpoints['time_point_id'].iloc[0]
            
            # Map to our location codes
            start_location = location_map.get(start_time_point, start_time_point)
            end_location = location_map.get(end_time_point, end_time_point)
            
            trip = {
                'half_trip_id': half_trip_id,
                'route_id': group['route_id'].iloc[0],
                'direction_id': group['direction_id'].iloc[0],
                'start_location': start_location,
                'end_location': end_location,
                'scheduled_start_time': startpoints['scheduled'].iloc[0],
                'scheduled_end_time': endpoints['scheduled'].iloc[0],
                'actual_start_time': startpoints['actual'].iloc[0],
                'actual_end_time': endpoints['actual'].iloc[0]
            }
            trips.append(trip)
    
    # Convert to DataFrame
    trips_df = pd.DataFrame(trips)
    
    # Parse datetime strings
    for col in ['scheduled_start_time', 'scheduled_end_time', 'actual_start_time', 'actual_end_time']:
        if col in trips_df.columns:
            # Convert times to datetime objects
            trips_df[col] = pd.to_datetime(trips_df[col], errors='coerce')
    
    return trips_df

def calculate_costs(trips_df, depot_id=DEPOT_ID):
    """
    Calculate costs between trips and depot connections using distances from Google Maps
    
    Args:
        trips_df: DataFrame of trip nodes
        depot_id: ID of the depot
        
    Returns:
        DataFrame with trip-to-trip costs and depot connections
    """
    # Get parameters
    cost_per_mile = BUS_PARAMS['cost_per_mile']
    cost_per_minute = BUS_PARAMS['cost_per_minute']
    waiting_cost_factor = BUS_PARAMS['waiting_cost_factor']
    delay_risk_factor = BUS_PARAMS['delay_risk_factor']
    
    # Create empty lists to store connections
    connections = []
    
    # Add depot to start connections
    for _, trip in trips_df.iterrows():
        # Get distance from depot to this trip's start location
        if trip['start_location'] == 'welst':
            distance = DISTANCES['depot_to_welst']
        elif trip['start_location'] == 'elmst':
            distance = DISTANCES['depot_to_elmst']
        else:
            # Use average distance for any other locations
            distance = (DISTANCES['depot_to_welst'] + DISTANCES['depot_to_elmst']) / 2
            
        # Calculate travel time
        travel_time = calculate_travel_time(distance)
        
        # Calculate costs
        deadheading_cost = distance * cost_per_mile + travel_time * cost_per_minute
        
        connection = {
            'from_id': depot_id,
            'to_id': trip['half_trip_id'],
            'from_type': 'depot',
            'to_type': 'trip',
            'from_location': depot_id,
            'to_location': trip['start_location'],
            'distance_miles': distance,
            'travel_time_minutes': travel_time,
            'deadheading_cost': deadheading_cost,
            'waiting_cost': 0,
            'delay_risk_cost': 0,
            'total_cost': deadheading_cost
        }
        connections.append(connection)
    
    # Add end to depot connections
    for _, trip in trips_df.iterrows():
        # Get distance from this trip's end location to depot
        if trip['end_location'] == 'welst':
            distance = DISTANCES['welst_to_depot']
        elif trip['end_location'] == 'elmst':
            distance = DISTANCES['elmst_to_depot']
        else:
            # Use average distance for any other locations
            distance = (DISTANCES['welst_to_depot'] + DISTANCES['elmst_to_depot']) / 2
            
        # Calculate travel time
        travel_time = calculate_travel_time(distance)
        
        # Calculate costs
        deadheading_cost = distance * cost_per_mile + travel_time * cost_per_minute
        
        connection = {
            'from_id': trip['half_trip_id'],
            'to_id': depot_id,
            'from_type': 'trip',
            'to_type': 'depot',
            'from_location': trip['end_location'],
            'to_location': depot_id,
            'distance_miles': distance,
            'travel_time_minutes': travel_time,
            'deadheading_cost': deadheading_cost,
            'waiting_cost': 0,
            'delay_risk_cost': 0,
            'total_cost': deadheading_cost
        }
        connections.append(connection)
    
    # Add trip-to-trip connections
    for i, trip_i in trips_df.iterrows():
        for j, trip_j in trips_df.iterrows():
            # Skip same trip
            if i == j:
                continue
                
            # Check if trip_j can follow trip_i (time compatibility)
            if pd.isnull(trip_i['scheduled_end_time']) or pd.isnull(trip_j['scheduled_start_time']):
                continue
            
            # Get distance between end of trip_i and start of trip_j
            from_loc = trip_i['end_location']
            to_loc = trip_j['start_location']
            
            if from_loc == 'elmst' and to_loc == 'welst':
                distance = DISTANCES['elmst_to_welst']
            elif from_loc == 'welst' and to_loc == 'elmst':
                distance = DISTANCES['welst_to_elmst']
            elif from_loc == to_loc:
                distance = 0.1  # Short distance if same location
            else:
                # Use average for any other combinations
                distance = (DISTANCES['elmst_to_welst'] + DISTANCES['welst_to_elmst']) / 2
            
            # Calculate travel time
            travel_time = calculate_travel_time(distance)
            
            # Calculate earliest possible arrival time at start of trip_j
            earliest_arrival = trip_i['scheduled_end_time'] + timedelta(minutes=travel_time)
            
            # Check if trip_j can be served after trip_i
            if earliest_arrival <= trip_j['scheduled_start_time']:
                # Calculate waiting time
                waiting_time = (trip_j['scheduled_start_time'] - earliest_arrival).total_seconds() / 60
                
                # Calculate delay risk (if trip_i has history of delays)
                if not pd.isnull(trip_i['actual_end_time']) and not pd.isnull(trip_i['scheduled_end_time']):
                    delay = (trip_i['actual_end_time'] - trip_i['scheduled_end_time']).total_seconds() / 60
                    delay_risk = max(0, delay)
                else:
                    delay_risk = 0
                
                # Calculate costs
                deadheading_cost = distance * cost_per_mile + travel_time * cost_per_minute
                waiting_cost = waiting_time * waiting_cost_factor * cost_per_minute
                delay_risk_cost = delay_risk * delay_risk_factor * cost_per_minute
                total_cost = deadheading_cost + waiting_cost + delay_risk_cost
                
                connection = {
                    'from_id': trip_i['half_trip_id'],
                    'to_id': trip_j['half_trip_id'],
                    'from_type': 'trip',
                    'to_type': 'trip',
                    'from_location': trip_i['end_location'],
                    'to_location': trip_j['start_location'],
                    'distance_miles': distance,
                    'travel_time_minutes': travel_time,
                    'waiting_time_minutes': waiting_time,
                    'deadheading_cost': deadheading_cost,
                    'waiting_cost': waiting_cost,
                    'delay_risk_cost': delay_risk_cost,
                    'total_cost': total_cost
                }
                connections.append(connection)
    
    # Convert to DataFrame
    connections_df = pd.DataFrame(connections)
    return connections_df

def solve_arc_flow_mdvsp(trips_df, connections_df, depot_id=DEPOT_ID, max_vehicles=10):
    """
    Solve the MDVSP using the arc-flow formulation
    
    Args:
        trips_df: DataFrame of trip nodes
        connections_df: DataFrame of connections between trips
        depot_id: ID of the depot
        max_vehicles: Maximum number of vehicles available
        
    Returns:
        Solution DataFrame with vehicle schedules
    """
    print("Solving MDVSP using arc-flow formulation...")
    start_time = time.time()
    
    # Create a list of all trips
    trips = trips_df['half_trip_id'].tolist()
    
    # Create a dictionary to store connections
    connections = {}
    for _, conn in connections_df.iterrows():
        from_id = conn['from_id']
        to_id = conn['to_id']
        total_cost = conn['total_cost']
        
        # Add connection to the dictionary
        if from_id not in connections:
            connections[from_id] = {}
        connections[from_id][to_id] = total_cost
    
    # Create the PuLP model
    model = pl.LpProblem("MDVSP_ArcFlow", pl.LpMinimize)
    
    # Create variables
    x = {}
    for _, conn in connections_df.iterrows():
        from_id = conn['from_id']
        to_id = conn['to_id']
        
        # Create binary variable for this connection
        x[(from_id, to_id)] = pl.LpVariable(f"x_{from_id}_{to_id}", cat='Binary')
    
    # Objective function: minimize total cost
    model += pl.lpSum([x[(conn['from_id'], conn['to_id'])] * conn['total_cost'] for _, conn in connections_df.iterrows()])
    
    # Constraint 1: Flow conservation for trips
    for trip in trips:
        # Inflow equals outflow for each trip
        inflow = pl.lpSum([x[(from_id, trip)] for from_id in connections if trip in connections[from_id]])
        outflow = pl.lpSum([x[(trip, to_id)] for to_id in connections.get(trip, {})])
        
        model += inflow == 1, f"Trip_{trip}_must_be_served"
        model += inflow == outflow, f"Flow_conservation_{trip}"
    
    # Constraint 2: Maximum number of vehicles
    model += pl.lpSum([x[(depot_id, to_id)] for to_id in connections.get(depot_id, {})]) <= max_vehicles, "Max_vehicles"
    
    # Solve the model
    solver = pl.PULP_CBC_CMD(msg=True, timeLimit=300)
    model.solve(solver)
    
    end_time = time.time()
    solution_time = end_time - start_time
    
    print(f"Status: {pl.LpStatus[model.status]}")
    print(f"Total cost: {pl.value(model.objective)}")
    print(f"Solution time: {solution_time:.2f} seconds")
    
    # Extract solution
    if model.status == pl.LpStatusOptimal:
        # Create a list to store vehicle schedules
        vehicle_schedules = []
        
        # Start with connections from depot
        for to_id in connections.get(depot_id, {}):
            if pl.value(x[(depot_id, to_id)]) > 0.5:  # If this connection is used
                # Start a new vehicle schedule
                schedule = [{'type': 'depot', 'id': depot_id, 'location': depot_id}]
                
                # Follow the path of this vehicle
                current_id = to_id
                while current_id != depot_id:
                    # Add this trip to the schedule
                    trip_info = trips_df[trips_df['half_trip_id'] == current_id].iloc[0].to_dict()
                    schedule.append({
                        'type': 'trip',
                        'id': current_id,
                        'start_location': trip_info['start_location'],
                        'end_location': trip_info['end_location'],
                        'start_time': trip_info['scheduled_start_time'],
                        'end_time': trip_info['scheduled_end_time'],
                        'route_id': trip_info['route_id'],
                        'direction_id': trip_info['direction_id']
                    })
                    
                    # Find the next trip/depot
                    next_id = None
                    for possible_next in connections.get(current_id, {}):
                        if pl.value(x[(current_id, possible_next)]) > 0.5:
                            next_id = possible_next
                            break
                    
                    if next_id is None:
                        print(f"Warning: No next trip found for {current_id}")
                        break
                        
                    current_id = next_id
                
                # Close the schedule with return to depot
                schedule.append({'type': 'depot', 'id': depot_id, 'location': depot_id})
                
                # Add this schedule to the list
                vehicle_schedules.append(schedule)
        
        # Convert to DataFrame for easier analysis
        schedule_rows = []
        for vehicle_id, schedule in enumerate(vehicle_schedules):
            for stop_num, stop in enumerate(schedule):
                row = {
                    'vehicle_id': vehicle_id + 1,
                    'stop_num': stop_num + 1,
                    'type': stop['type'],
                    'id': stop['id']
                }
                
                # Add location information
                if 'location' in stop:
                    row['location'] = stop['location']
                
                # Add trip details if this is a trip
                if stop['type'] == 'trip':
                    row.update({
                        'start_location': stop['start_location'],
                        'end_location': stop['end_location'],
                        'start_time': stop['start_time'],
                        'end_time': stop['end_time'],
                        'route_id': stop['route_id'],
                        'direction_id': stop['direction_id']
                    })
                
                schedule_rows.append(row)
        
        schedules_df = pd.DataFrame(schedule_rows)
        return schedules_df
    else:
        print("No optimal solution found.")
        return None

def solve_set_partitioning_mdvsp(trips_df, connections_df, depot_id='fellsway_garage', max_vehicles=10):
    """
    Solve the MDVSP using the set partitioning formulation
    
    Args:
        trips_df: DataFrame of trip nodes
        connections_df: DataFrame of connections between trips
        depot_id: ID of the depot
        max_vehicles: Maximum number of vehicles available
        
    Returns:
        Solution DataFrame with vehicle schedules
    """
    print("Solving MDVSP using set partitioning formulation...")
    start_time = time.time()
    
    # Create a list of all trips
    trips = trips_df['half_trip_id'].tolist()
    
    # Create a graph representation for the network
    graph = {}
    for _, conn in connections_df.iterrows():
        from_id = conn['from_id']
        to_id = conn['to_id']
        cost = conn['total_cost']
        
        if from_id not in graph:
            graph[from_id] = {}
        graph[from_id][to_id] = cost
    
    # Generate all feasible vehicle schedules (paths from depot to depot)
    print("Generating feasible vehicle schedules...")
    feasible_schedules = []
    
    # Use a recursive function to find all paths from depot to depot
    def find_paths(current_path, visited_trips, current_cost):
        current_node = current_path[-1]
        
        # If we've returned to the depot and visited at least one trip, we have a feasible schedule
        if current_node == depot_id and len(current_path) > 2:
            # Extract just the trip IDs (exclude depot at start and end)
            trip_ids = [node for node in current_path[1:-1]]
            feasible_schedules.append({
                'trips': trip_ids,
                'cost': current_cost,
                'path': current_path.copy()  # Keep full path for reference
            })
            return
        
        # If we're at the maximum path length, stop recursion
        if len(current_path) > len(trips) + 2:
            return
        
        # Try all possible next nodes
        if current_node in graph:
            for next_node, edge_cost in graph[current_node].items():
                # Skip if this trip has already been visited
                if next_node in trips and next_node in visited_trips:
                    continue
                
                # Add this node to the path
                new_path = current_path + [next_node]
                new_visited = visited_trips.copy()
                if next_node in trips:
                    new_visited.add(next_node)
                
                # Continue recursion
                find_paths(new_path, new_visited, current_cost + edge_cost)
    
    # Start with paths from the depot
    find_paths([depot_id], set(), 0)
    
    print(f"Generated {len(feasible_schedules)} feasible schedules")
    
    # Create the PuLP model
    model = pl.LpProblem("MDVSP_SetPartitioning", pl.LpMinimize)
    
    # Create variables - one for each feasible schedule
    y = {}
    for i, schedule in enumerate(feasible_schedules):
        y[i] = pl.LpVariable(f"y_{i}", cat='Binary')
    
    # Objective function: minimize total cost
    model += pl.lpSum([y[i] * schedule['cost'] for i, schedule in enumerate(feasible_schedules)])
    
    # Constraint 1: Each trip must be covered exactly once
    for trip in trips:
        model += pl.lpSum([y[i] for i, schedule in enumerate(feasible_schedules) if trip in schedule['trips']]) == 1, f"Trip_{trip}_must_be_served"
    
    # Constraint 2: Maximum number of vehicles
    model += pl.lpSum([y[i] for i in range(len(feasible_schedules))]) <= max_vehicles, "Max_vehicles"
    
    # Solve the model
    solver = pl.PULP_CBC_CMD(msg=True, timeLimit=300)
    model.solve(solver)
    
    end_time = time.time()
    solution_time = end_time - start_time
    
    print(f"Status: {pl.LpStatus[model.status]}")
    print(f"Total cost: {pl.value(model.objective)}")
    print(f"Solution time: {solution_time:.2f} seconds")
    
    # Extract solution
    if model.status == pl.LpStatusOptimal:
        # Create a list to store vehicle schedules
        schedule_rows = []
        
        # Process each selected schedule
        vehicle_id = 1
        for i, schedule in enumerate(feasible_schedules):
            if pl.value(y[i]) > 0.5:  # If this schedule is used
                path = schedule['path']
                
                # Add depot departure
                schedule_rows.append({
                    'vehicle_id': vehicle_id,
                    'stop_num': 1,
                    'type': 'depot',
                    'id': depot_id,
                    'location': depot_id
                })
                
                # Add each trip in the schedule
                for stop_num, trip_id in enumerate(schedule['trips']):
                    # Get trip details
                    trip_info = trips_df[trips_df['half_trip_id'] == trip_id].iloc[0].to_dict()
                    
                    schedule_rows.append({
                        'vehicle_id': vehicle_id,
                        'stop_num': stop_num + 2,
                        'type': 'trip',
                        'id': trip_id,
                        'start_location': trip_info['start_location'],
                        'end_location': trip_info['end_location'],
                        'start_time': trip_info['scheduled_start_time'],
                        'end_time': trip_info['scheduled_end_time'],
                        'route_id': trip_info['route_id'],
                        'direction_id': trip_info['direction_id']
                    })
                
                # Add depot return
                schedule_rows.append({
                    'vehicle_id': vehicle_id,
                    'stop_num': len(schedule['trips']) + 2,
                    'type': 'depot',
                    'id': depot_id,
                    'location': depot_id
                })
                
                vehicle_id += 1
        
        schedules_df = pd.DataFrame(schedule_rows)
        return schedules_df
    else:
        print("No optimal solution found.")
        return None

def analyze_lp_relaxation_set_partitioning(trips_df, connections_df, depot_id='fellsway_garage', max_vehicles=10):
    """
    Analyze the LP relaxation of set partitioning formulation
    
    Args:
        trips_df: DataFrame of trip nodes
        connections_df: DataFrame of connections between trips
        depot_id: ID of the depot
        max_vehicles: Maximum number of vehicles available
    """
    print("Analyzing LP relaxation of set partitioning formulation...")
    start_time = time.time()
    
    # Create a list of all trips
    trips = trips_df['half_trip_id'].tolist()
    
    # Create a graph representation for the network
    graph = {}
    for _, conn in connections_df.iterrows():
        from_id = conn['from_id']
        to_id = conn['to_id']
        cost = conn['total_cost']
        
        if from_id not in graph:
            graph[from_id] = {}
        graph[from_id][to_id] = cost
    
    # Generate all feasible vehicle schedules
    feasible_schedules = []
    
    # Use a recursive function to find all paths from depot to depot
    def find_paths(current_path, visited_trips, current_cost):
        current_node = current_path[-1]
        
        # If we've returned to the depot and visited at least one trip, we have a feasible schedule
        if current_node == depot_id and len(current_path) > 2:
            # Extract just the trip IDs (exclude depot at start and end)
            trip_ids = [node for node in current_path[1:-1]]
            feasible_schedules.append({
                'trips': trip_ids,
                'cost': current_cost
            })
            return
        
        # If we're at the maximum path length, stop recursion
        if len(current_path) > len(trips) + 2:
            return
        
        # Try all possible next nodes
        if current_node in graph:
            for next_node, edge_cost in graph[current_node].items():
                # Skip if this trip has already been visited
                if next_node in trips and next_node in visited_trips:
                    continue
                
                # Add this node to the path
                new_path = current_path + [next_node]
                new_visited = visited_trips.copy()
                if next_node in trips:
                    new_visited.add(next_node)
                
                # Continue recursion
                find_paths(new_path, new_visited, current_cost + edge_cost)
    
    # Start with paths from the depot
    find_paths([depot_id], set(), 0)
    
    print(f"Generated {len(feasible_schedules)} feasible schedules")
    
    # Create the PuLP model
    model = pl.LpProblem("MDVSP_SetPartitioning_LP", pl.LpMinimize)
    
    # Create continuous variables (for LP relaxation)
    y = {}
    for i, schedule in enumerate(feasible_schedules):
        y[i] = pl.LpVariable(f"y_{i}", lowBound=0, upBound=1, cat='Continuous')
    
    # Objective function: minimize total cost
    model += pl.lpSum([y[i] * schedule['cost'] for i, schedule in enumerate(feasible_schedules)])
    
    # Constraint 1: Each trip must be covered exactly once
    for trip in trips:
        model += pl.lpSum([y[i] for i, schedule in enumerate(feasible_schedules) if trip in schedule['trips']]) == 1, f"Trip_{trip}_must_be_served"
    
    # Constraint 2: Maximum number of vehicles
    model += pl.lpSum([y[i] for i in range(len(feasible_schedules))]) <= max_vehicles, "Max_vehicles"
    
    # Solve the LP relaxation
    solver = pl.PULP_CBC_CMD(msg=False)
    model.solve(solver)
    
    end_time = time.time()
    solution_time = end_time - start_time
    
    print(f"Set partitioning LP relaxation - Status: {pl.LpStatus[model.status]}")
    print(f"Set partitioning LP relaxation - Objective: {pl.value(model.objective)}")
    print(f"Solution time: {solution_time:.2f} seconds")

def analyze_lp_relaxation(trips_df, connections_df, depot_id=DEPOT_ID, max_vehicles=10):
    """
    Analyze the LP relaxation of arc-flow formulation
    
    Args:
        trips_df: DataFrame of trip nodes
        connections_df: DataFrame of connections between trips
        depot_id: ID of the depot
        max_vehicles: Maximum number of vehicles available
    """
    print("Analyzing LP relaxation of arc-flow formulation...")
    
    # Create a list of all trips
    trips = trips_df['half_trip_id'].tolist()
    
    # Create a dictionary to store connections
    connections = {}
    for _, conn in connections_df.iterrows():
        from_id = conn['from_id']
        to_id = conn['to_id']
        total_cost = conn['total_cost']
        
        # Add connection to the dictionary
        if from_id not in connections:
            connections[from_id] = {}
        connections[from_id][to_id] = total_cost
    
    # Create the PuLP model for arc-flow
    arc_flow_model = pl.LpProblem("MDVSP_ArcFlow_LP", pl.LpMinimize)
    
    # Create continuous variables
    x = {}
    for _, conn in connections_df.iterrows():
        from_id = conn['from_id']
        to_id = conn['to_id']
        
        # Create continuous variable for this connection
        x[(from_id, to_id)] = pl.LpVariable(f"x_{from_id}_{to_id}", lowBound=0, upBound=1, cat='Continuous')
    
    # Objective function: minimize total cost
    arc_flow_model += pl.lpSum([x[(conn['from_id'], conn['to_id'])] * conn['total_cost'] for _, conn in connections_df.iterrows()])
    
    # Constraint 1: Flow conservation for trips
    for trip in trips:
        # Inflow equals outflow for each trip
        inflow = pl.lpSum([x[(from_id, trip)] for from_id in connections if trip in connections[from_id]])
        outflow = pl.lpSum([x[(trip, to_id)] for to_id in connections.get(trip, {})])
        
        arc_flow_model += inflow == 1, f"Trip_{trip}_must_be_served"
        arc_flow_model += inflow == outflow, f"Flow_conservation_{trip}"
    
    # Constraint 2: Maximum number of vehicles
    arc_flow_model += pl.lpSum([x[(depot_id, to_id)] for to_id in connections.get(depot_id, {})]) <= max_vehicles, "Max_vehicles"
    
    # Solve the LP relaxation
    solver = pl.PULP_CBC_CMD(msg=False)
    arc_flow_model.solve(solver)
    
    print(f"Arc-flow LP relaxation - Status: {pl.LpStatus[arc_flow_model.status]}")
    print(f"Arc-flow LP relaxation - Objective: {pl.value(arc_flow_model.objective)}")

def main():
    # Input and output paths
    input_csv = 'filtered_route100_15h_20h.csv'
    trip_nodes_output = 'route100_15h_20h_trip_nodes_pulp.csv'
    connections_output = 'route100_15h_20h_connections_pulp.csv'
    arc_flow_solution_output = 'route100_15h_20h_arc_flow_solution_pulp.csv'
    set_partitioning_solution_output = 'route100_15h_20h_set_partitioning_solution_pulp.csv'
    
    try:
        # Step 1: Create trip nodes
        print("\n--- STEP 1: Creating trip nodes ---")
        trips_df = create_trip_nodes(input_csv)
        
        # Print summary
        print(f"\nCreated {len(trips_df)} trip nodes")
        if not trips_df.empty:
            print("\nSample trip nodes:")
            print(trips_df.head())
            
            # Save to CSV
            trips_df.to_csv(trip_nodes_output, index=False)
            print(f"\nTrip nodes saved to {trip_nodes_output}")
        
        # Step 2: Calculate connections and costs
        print("\n--- STEP 2: Calculating connections and costs ---")
        connections_df = calculate_costs(trips_df)
        
        # Print summary
        print(f"\nCreated {len(connections_df)} connections")
        print("\nSample connections:")
        print(connections_df.head())
        
        # Save to CSV
        connections_df.to_csv(connections_output, index=False)
        print(f"\nConnections saved to {connections_output}")
        
        # Step 3: Analyze LP relaxation of arc-flow formulation
        print("\n--- STEP 3: Analyzing LP relaxation of arc-flow formulation ---")
        analyze_lp_relaxation(trips_df, connections_df)
        
        # Step 4: Analyze LP relaxation of set partitioning formulation
        print("\n--- STEP 4: Analyzing LP relaxation of set partitioning formulation ---")
        analyze_lp_relaxation_set_partitioning(trips_df, connections_df)
        
        # Step 5: Solve using arc-flow formulation
        print("\n--- STEP 5: Solving with arc-flow formulation ---")
        arc_flow_solution = solve_arc_flow_mdvsp(trips_df, connections_df)
        
        if arc_flow_solution is not None:
            print("\nSample arc-flow solution:")
            print(arc_flow_solution.head())
            
            # Save to CSV
            arc_flow_solution.to_csv(arc_flow_solution_output, index=False)
            print(f"\nArc-flow solution saved to {arc_flow_solution_output}")
        
        # Step 6: Solve using set partitioning formulation
        print("\n--- STEP 6: Solving with set partitioning formulation ---")
        set_partitioning_solution = solve_set_partitioning_mdvsp(trips_df, connections_df)
        
        if set_partitioning_solution is not None:
            print("\nSample set partitioning solution:")
            print(set_partitioning_solution.head())
            
            # Save to CSV
            set_partitioning_solution.to_csv(set_partitioning_solution_output, index=False)
            print(f"\nSet partitioning solution saved to {set_partitioning_solution_output}")
    
    except Exception as e:
        print(f"Error: {e}")

if __name__ == "__main__":
    main()


--- STEP 1: Creating trip nodes ---
Total records: 48
Unique half_trip_ids: 24
Unique stop_ids: [52711  8301  8302 52720]
Unique time_point_ids: ['welst' 'elm']

Created 24 trip nodes

Sample trip nodes:
   half_trip_id  route_id direction_id start_location end_location  \
0    65802625.0       100     Outbound          welst          elm   
1    65802626.0       100      Inbound            elm        welst   
2    65802627.0       100     Outbound          welst          elm   
3    65802628.0       100      Inbound            elm        welst   
4    65802629.0       100     Outbound          welst          elm   

  scheduled_start_time  scheduled_end_time   actual_start_time  \
0  2025-04-29 15:00:00 2025-04-29 15:11:00 2025-04-29 15:06:11   
1  2025-04-29 15:15:00 2025-04-29 15:26:00 2025-04-29 15:24:30   
2  2025-04-29 15:30:00 2025-04-29 15:41:00 2025-04-29 15:38:30   
3  2025-04-29 15:45:00 2025-04-29 15:57:00 2025-04-29 15:48:49   
4  2025-04-29 16:00:00 2025-04-29 16:11:00 2

/var/folders/v8/1793d44577q9lqgx_01s8skm0000gn/T/ipykernel_18434/2398953751.py:87: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  trips_df[col] = pd.to_datetime(trips_df[col], errors='coerce')
/var/folders/v8/1793d44577q9lqgx_01s8skm0000gn/T/ipykernel_18434/2398953751.py:87: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  trips_df[col] = pd.to_datetime(trips_df[col], errors='coerce')
/var/folders/v8/1793d44577q9lqgx_01s8skm0000gn/T/ipykernel_18434/2398953751.py:87: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  trips_df[col] = pd.to_datetime(trips_df[col], errors='coerce')
/var

Arc-flow LP relaxation - Status: Optimal
Arc-flow LP relaxation - Objective: 378.29999999999995

--- STEP 4: Analyzing LP relaxation of set partitioning formulation ---
Analyzing LP relaxation of set partitioning formulation...
Generated 16777215 feasible schedules


# 2. Using Gurobi for multiple time slot

## a. 3 hours

In [5]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import gurobipy as gp
from gurobipy import GRB
import time

# Define bus operation parameters
BUS_PARAMS = {
    'avg_speed_mph': 12,  # Average bus speed in mph
    'cost_per_mile': 2.5,  # Operating cost per mile
    'cost_per_minute': 1.0,  # Cost per minute of operation
    'waiting_cost_factor': 0.5,  # Cost factor for waiting time
    'delay_risk_factor': 2.0  # Cost factor for delay risk
}

# Define the depot
DEPOT_ID = 'fellsway_garage'

# Define distances from Google Maps screenshots (in miles)
DISTANCES = {
    'depot_to_welst': 2.0,  # Fellsway Garage to Wellington St
    'depot_to_elmst': 1.4,  # Fellsway Garage to Elm St
    'welst_to_depot': 2.0,  # Wellington St to Fellsway Garage
    'elmst_to_depot': 1.4,  # Elm St to Fellsway Garage
    'elmst_to_welst': 1.5,  # Estimated distance between stops
    'welst_to_elmst': 1.5   # Estimated distance between stops
}

def calculate_travel_time(distance, speed=BUS_PARAMS['avg_speed_mph']):
    """Calculate travel time based on distance and speed"""
    return (distance / speed) * 60

def create_trip_nodes(csv_path):
    """Create trip nodes from the filtered CSV data"""
    # Read the filtered data
    df = pd.read_csv(csv_path)
    
    # Print data summary
    print(f"Total records: {len(df)}")
    print(f"Unique half_trip_ids: {df['half_trip_id'].nunique()}")
    print(f"Unique stop_ids: {df['stop_id'].unique()}")
    print(f"Unique time_point_ids: {df['time_point_id'].unique()}")
    
    # Map time_point_ids to our location codes
    location_map = {
        'elmst': 'elmst',
        'welst': 'welst'
    }
    
    # Group by half_trip_id
    trips = []
    
    # For each half_trip_id, find its startpoint and endpoint
    for half_trip_id, group in df.groupby('half_trip_id'):
        startpoints = group[group['point_type'] == 'Startpoint']
        endpoints = group[group['point_type'] == 'Endpoint']
        
        if not startpoints.empty and not endpoints.empty:
            # Get start and end locations
            start_time_point = startpoints['time_point_id'].iloc[0]
            end_time_point = endpoints['time_point_id'].iloc[0]
            
            # Map to our location codes
            start_location = location_map.get(start_time_point, start_time_point)
            end_location = location_map.get(end_time_point, end_time_point)
            
            trip = {
                'half_trip_id': half_trip_id,
                'route_id': group['route_id'].iloc[0],
                'direction_id': group['direction_id'].iloc[0],
                'start_location': start_location,
                'end_location': end_location,
                'scheduled_start_time': startpoints['scheduled'].iloc[0],
                'scheduled_end_time': endpoints['scheduled'].iloc[0],
                'actual_start_time': startpoints['actual'].iloc[0],
                'actual_end_time': endpoints['actual'].iloc[0]
            }
            trips.append(trip)
    
    # Convert to DataFrame
    trips_df = pd.DataFrame(trips)
    
    # Parse datetime strings
    for col in ['scheduled_start_time', 'scheduled_end_time', 'actual_start_time', 'actual_end_time']:
        if col in trips_df.columns:
            # Convert times to datetime objects
            trips_df[col] = pd.to_datetime(trips_df[col], errors='coerce')
    
    return trips_df

def calculate_costs(trips_df, depot_id=DEPOT_ID):
    """
    Calculate costs between trips and depot connections using distances from Google Maps
    
    Args:
        trips_df: DataFrame of trip nodes
        depot_id: ID of the depot
        
    Returns:
        DataFrame with trip-to-trip costs and depot connections
    """
    # Get parameters
    cost_per_mile = BUS_PARAMS['cost_per_mile']
    cost_per_minute = BUS_PARAMS['cost_per_minute']
    waiting_cost_factor = BUS_PARAMS['waiting_cost_factor']
    delay_risk_factor = BUS_PARAMS['delay_risk_factor']
    
    # Create empty lists to store connections
    connections = []
    
    # Add depot to start connections
    for _, trip in trips_df.iterrows():
        # Get distance from depot to this trip's start location
        if trip['start_location'] == 'welst':
            distance = DISTANCES['depot_to_welst']
        elif trip['start_location'] == 'elmst':
            distance = DISTANCES['depot_to_elmst']
        else:
            # Use average distance for any other locations
            distance = (DISTANCES['depot_to_welst'] + DISTANCES['depot_to_elmst']) / 2
            
        # Calculate travel time
        travel_time = calculate_travel_time(distance)
        
        # Calculate costs
        deadheading_cost = distance * cost_per_mile + travel_time * cost_per_minute
        
        connection = {
            'from_id': depot_id,
            'to_id': trip['half_trip_id'],
            'from_type': 'depot',
            'to_type': 'trip',
            'from_location': depot_id,
            'to_location': trip['start_location'],
            'distance_miles': distance,
            'travel_time_minutes': travel_time,
            'deadheading_cost': deadheading_cost,
            'waiting_cost': 0,
            'delay_risk_cost': 0,
            'total_cost': deadheading_cost
        }
        connections.append(connection)
    
    # Add end to depot connections
    for _, trip in trips_df.iterrows():
        # Get distance from this trip's end location to depot
        if trip['end_location'] == 'welst':
            distance = DISTANCES['welst_to_depot']
        elif trip['end_location'] == 'elmst':
            distance = DISTANCES['elmst_to_depot']
        else:
            # Use average distance for any other locations
            distance = (DISTANCES['welst_to_depot'] + DISTANCES['elmst_to_depot']) / 2
            
        # Calculate travel time
        travel_time = calculate_travel_time(distance)
        
        # Calculate costs
        deadheading_cost = distance * cost_per_mile + travel_time * cost_per_minute
        
        connection = {
            'from_id': trip['half_trip_id'],
            'to_id': depot_id,
            'from_type': 'trip',
            'to_type': 'depot',
            'from_location': trip['end_location'],
            'to_location': depot_id,
            'distance_miles': distance,
            'travel_time_minutes': travel_time,
            'deadheading_cost': deadheading_cost,
            'waiting_cost': 0,
            'delay_risk_cost': 0,
            'total_cost': deadheading_cost
        }
        connections.append(connection)
    
    # Add trip-to-trip connections
    for i, trip_i in trips_df.iterrows():
        for j, trip_j in trips_df.iterrows():
            # Skip same trip
            if i == j:
                continue
                
            # Check if trip_j can follow trip_i (time compatibility)
            if pd.isnull(trip_i['scheduled_end_time']) or pd.isnull(trip_j['scheduled_start_time']):
                continue
            
            # Get distance between end of trip_i and start of trip_j
            from_loc = trip_i['end_location']
            to_loc = trip_j['start_location']
            
            if from_loc == 'elmst' and to_loc == 'welst':
                distance = DISTANCES['elmst_to_welst']
            elif from_loc == 'welst' and to_loc == 'elmst':
                distance = DISTANCES['welst_to_elmst']
            elif from_loc == to_loc:
                distance = 0.1  # Short distance if same location
            else:
                # Use average for any other combinations
                distance = (DISTANCES['elmst_to_welst'] + DISTANCES['welst_to_elmst']) / 2
            
            # Calculate travel time
            travel_time = calculate_travel_time(distance)
            
            # Calculate earliest possible arrival time at start of trip_j
            earliest_arrival = trip_i['scheduled_end_time'] + timedelta(minutes=travel_time)
            
            # Check if trip_j can be served after trip_i
            if earliest_arrival <= trip_j['scheduled_start_time']:
                # Calculate waiting time
                waiting_time = (trip_j['scheduled_start_time'] - earliest_arrival).total_seconds() / 60
                
                # Calculate delay risk (if trip_i has history of delays)
                if not pd.isnull(trip_i['actual_end_time']) and not pd.isnull(trip_i['scheduled_end_time']):
                    delay = (trip_i['actual_end_time'] - trip_i['scheduled_end_time']).total_seconds() / 60
                    delay_risk = max(0, delay)
                else:
                    delay_risk = 0
                
                # Calculate costs
                deadheading_cost = distance * cost_per_mile + travel_time * cost_per_minute
                waiting_cost = waiting_time * waiting_cost_factor * cost_per_minute
                delay_risk_cost = delay_risk * delay_risk_factor * cost_per_minute
                total_cost = deadheading_cost + waiting_cost + delay_risk_cost
                
                connection = {
                    'from_id': trip_i['half_trip_id'],
                    'to_id': trip_j['half_trip_id'],
                    'from_type': 'trip',
                    'to_type': 'trip',
                    'from_location': trip_i['end_location'],
                    'to_location': trip_j['start_location'],
                    'distance_miles': distance,
                    'travel_time_minutes': travel_time,
                    'waiting_time_minutes': waiting_time,
                    'deadheading_cost': deadheading_cost,
                    'waiting_cost': waiting_cost,
                    'delay_risk_cost': delay_risk_cost,
                    'total_cost': total_cost
                }
                connections.append(connection)
    
    # Convert to DataFrame
    connections_df = pd.DataFrame(connections)
    return connections_df

def solve_arc_flow_mdvsp(trips_df, connections_df, depot_id=DEPOT_ID, max_vehicles=10):
    """
    Solve the MDVSP using the arc-flow formulation with Gurobi
    
    Args:
        trips_df: DataFrame of trip nodes
        connections_df: DataFrame of connections between trips
        depot_id: ID of the depot
        max_vehicles: Maximum number of vehicles available
        
    Returns:
        Solution DataFrame with vehicle schedules
    """
    print("Solving MDVSP using arc-flow formulation with Gurobi...")
    start_time = time.time()
    
    # Create a list of all trips
    trips = trips_df['half_trip_id'].tolist()
    
    # Create a dictionary to store connections
    connections = {}
    for _, conn in connections_df.iterrows():
        from_id = conn['from_id']
        to_id = conn['to_id']
        total_cost = conn['total_cost']
        
        # Add connection to the dictionary
        if from_id not in connections:
            connections[from_id] = {}
        connections[from_id][to_id] = total_cost
    
    # Create the Gurobi model
    model = gp.Model("MDVSP_ArcFlow")
    
    # Create variables
    x = {}
    for _, conn in connections_df.iterrows():
        from_id = conn['from_id']
        to_id = conn['to_id']
        
        # Create binary variable for this connection
        x[(from_id, to_id)] = model.addVar(vtype=GRB.BINARY, name=f"x_{from_id}_{to_id}")
    
    # Update the model to include new variables
    model.update()
    
    # Objective function: minimize total cost
    obj = gp.LinExpr()
    for _, conn in connections_df.iterrows():
        from_id = conn['from_id']
        to_id = conn['to_id']
        obj += x[(from_id, to_id)] * conn['total_cost']
    
    model.setObjective(obj, GRB.MINIMIZE)
    
    # Constraint 1: Flow conservation for trips
    for trip in trips:
        # Inflow equals outflow for each trip
        inflow = gp.LinExpr()
        outflow = gp.LinExpr()
        
        for from_id in connections:
            if trip in connections[from_id]:
                inflow += x[(from_id, trip)]
        
        if trip in connections:
            for to_id in connections[trip]:
                outflow += x[(trip, to_id)]
        
        model.addConstr(inflow == 1, f"Trip_{trip}_must_be_served")
        model.addConstr(inflow == outflow, f"Flow_conservation_{trip}")
    
    # Constraint 2: Maximum number of vehicles
    outflow_from_depot = gp.LinExpr()
    if depot_id in connections:
        for to_id in connections[depot_id]:
            outflow_from_depot += x[(depot_id, to_id)]
    
    model.addConstr(outflow_from_depot <= max_vehicles, "Max_vehicles")
    
    # Set time limit and solver parameters
    model.setParam('TimeLimit', 300)  # 5 minutes time limit
    model.setParam('OutputFlag', 1)   # Display solver output
    
    # Solve the model
    model.optimize()
    
    end_time = time.time()
    solution_time = end_time - start_time
    
    print(f"Status: {model.status}")
    if model.status == GRB.OPTIMAL or model.status == GRB.TIME_LIMIT:
        print(f"Total cost: {model.objVal}")
    print(f"Solution time: {solution_time:.2f} seconds")
    
    # Extract solution
    if model.status == GRB.OPTIMAL or model.status == GRB.TIME_LIMIT:
        # Create a list to store vehicle schedules
        vehicle_schedules = []
        
        # Start with connections from depot
        if depot_id in connections:
            for to_id in connections[depot_id]:
                if x[(depot_id, to_id)].x > 0.5:  # If this connection is used
                    # Start a new vehicle schedule
                    schedule = [{'type': 'depot', 'id': depot_id, 'location': depot_id}]
                    
                    # Follow the path of this vehicle
                    current_id = to_id
                    while current_id != depot_id:
                        # Add this trip to the schedule
                        trip_info = trips_df[trips_df['half_trip_id'] == current_id].iloc[0].to_dict()
                        schedule.append({
                            'type': 'trip',
                            'id': current_id,
                            'start_location': trip_info['start_location'],
                            'end_location': trip_info['end_location'],
                            'start_time': trip_info['scheduled_start_time'],
                            'end_time': trip_info['scheduled_end_time'],
                            'route_id': trip_info['route_id'],
                            'direction_id': trip_info['direction_id']
                        })
                        
                        # Find the next trip/depot
                        next_id = None
                        if current_id in connections:
                            for possible_next in connections[current_id]:
                                if x[(current_id, possible_next)].x > 0.5:
                                    next_id = possible_next
                                    break
                        
                        if next_id is None:
                            print(f"Warning: No next trip found for {current_id}")
                            break
                            
                        current_id = next_id
                    
                    # Close the schedule with return to depot
                    schedule.append({'type': 'depot', 'id': depot_id, 'location': depot_id})
                    
                    # Add this schedule to the list
                    vehicle_schedules.append(schedule)
        
        # Convert to DataFrame for easier analysis
        schedule_rows = []
        for vehicle_id, schedule in enumerate(vehicle_schedules):
            for stop_num, stop in enumerate(schedule):
                row = {
                    'vehicle_id': vehicle_id + 1,
                    'stop_num': stop_num + 1,
                    'type': stop['type'],
                    'id': stop['id']
                }
                
                # Add location information
                if 'location' in stop:
                    row['location'] = stop['location']
                
                # Add trip details if this is a trip
                if stop['type'] == 'trip':
                    row.update({
                        'start_location': stop['start_location'],
                        'end_location': stop['end_location'],
                        'start_time': stop['start_time'],
                        'end_time': stop['end_time'],
                        'route_id': stop['route_id'],
                        'direction_id': stop['direction_id']
                    })
                
                schedule_rows.append(row)
        
        schedules_df = pd.DataFrame(schedule_rows)
        return schedules_df
    else:
        print("No optimal solution found.")
        return None

def solve_set_partitioning_mdvsp(trips_df, connections_df, depot_id='fellsway_garage', max_vehicles=10):
    """
    Solve the MDVSP using the set partitioning formulation with Gurobi
    
    Args:
        trips_df: DataFrame of trip nodes
        connections_df: DataFrame of connections between trips
        depot_id: ID of the depot
        max_vehicles: Maximum number of vehicles available
        
    Returns:
        Solution DataFrame with vehicle schedules
    """
    print("Solving MDVSP using set partitioning formulation with Gurobi...")
    start_time = time.time()
    
    # Create a list of all trips
    trips = trips_df['half_trip_id'].tolist()
    
    # Create a graph representation for the network
    graph = {}
    for _, conn in connections_df.iterrows():
        from_id = conn['from_id']
        to_id = conn['to_id']
        cost = conn['total_cost']
        
        if from_id not in graph:
            graph[from_id] = {}
        graph[from_id][to_id] = cost
    
    # Generate all feasible vehicle schedules (paths from depot to depot)
    print("Generating feasible vehicle schedules...")
    feasible_schedules = []
    
    # Use a recursive function to find all paths from depot to depot
    def find_paths(current_path, visited_trips, current_cost):
        current_node = current_path[-1]
        
        # If we've returned to the depot and visited at least one trip, we have a feasible schedule
        if current_node == depot_id and len(current_path) > 2:
            # Extract just the trip IDs (exclude depot at start and end)
            trip_ids = [node for node in current_path[1:-1]]
            feasible_schedules.append({
                'trips': trip_ids,
                'cost': current_cost,
                'path': current_path.copy()  # Keep full path for reference
            })
            return
        
        # If we're at the maximum path length, stop recursion
        if len(current_path) > len(trips) + 2:
            return
        
        # Try all possible next nodes
        if current_node in graph:
            for next_node, edge_cost in graph[current_node].items():
                # Skip if this trip has already been visited
                if next_node in trips and next_node in visited_trips:
                    continue
                
                # Add this node to the path
                new_path = current_path + [next_node]
                new_visited = visited_trips.copy()
                if next_node in trips:
                    new_visited.add(next_node)
                
                # Continue recursion
                find_paths(new_path, new_visited, current_cost + edge_cost)
    
    # Start with paths from the depot
    find_paths([depot_id], set(), 0)
    
    print(f"Generated {len(feasible_schedules)} feasible schedules")
    
    # Create the Gurobi model
    model = gp.Model("MDVSP_SetPartitioning")
    
    # Create variables - one for each feasible schedule
    y = {}
    for i, schedule in enumerate(feasible_schedules):
        y[i] = model.addVar(vtype=GRB.BINARY, name=f"y_{i}")
    
    # Update model to incorporate new variables
    model.update()
    
    # Objective function: minimize total cost
    obj = gp.LinExpr()
    for i, schedule in enumerate(feasible_schedules):
        obj += y[i] * schedule['cost']
    
    model.setObjective(obj, GRB.MINIMIZE)
    
    # Constraint 1: Each trip must be covered exactly once
    for trip in trips:
        constr = gp.LinExpr()
        for i, schedule in enumerate(feasible_schedules):
            if trip in schedule['trips']:
                constr += y[i]
        model.addConstr(constr == 1, f"Trip_{trip}_must_be_served")
    
    # Constraint 2: Maximum number of vehicles
    vehicles_constr = gp.LinExpr()
    for i in range(len(feasible_schedules)):
        vehicles_constr += y[i]
    model.addConstr(vehicles_constr <= max_vehicles, "Max_vehicles")
    
    # Set time limit and solver parameters
    model.setParam('TimeLimit', 300)  # 5 minutes time limit
    model.setParam('OutputFlag', 1)   # Display solver output
    
    # Solve the model
    model.optimize()
    
    end_time = time.time()
    solution_time = end_time - start_time
    
    print(f"Status: {model.status}")
    if model.status == GRB.OPTIMAL or model.status == GRB.TIME_LIMIT:
        print(f"Total cost: {model.objVal}")
    print(f"Solution time: {solution_time:.2f} seconds")
    
    # Extract solution
    if model.status == GRB.OPTIMAL or model.status == GRB.TIME_LIMIT:
        # Create a list to store vehicle schedules
        schedule_rows = []
        
        # Process each selected schedule
        vehicle_id = 1
        for i, schedule in enumerate(feasible_schedules):
            if y[i].x > 0.5:  # If this schedule is used
                path = schedule['path']
                
                # Add depot departure
                schedule_rows.append({
                    'vehicle_id': vehicle_id,
                    'stop_num': 1,
                    'type': 'depot',
                    'id': depot_id,
                    'location': depot_id
                })
                
                # Add each trip in the schedule
                for stop_num, trip_id in enumerate(schedule['trips']):
                    # Get trip details
                    trip_info = trips_df[trips_df['half_trip_id'] == trip_id].iloc[0].to_dict()
                    
                    schedule_rows.append({
                        'vehicle_id': vehicle_id,
                        'stop_num': stop_num + 2,
                        'type': 'trip',
                        'id': trip_id,
                        'start_location': trip_info['start_location'],
                        'end_location': trip_info['end_location'],
                        'start_time': trip_info['scheduled_start_time'],
                        'end_time': trip_info['scheduled_end_time'],
                        'route_id': trip_info['route_id'],
                        'direction_id': trip_info['direction_id']
                    })
                
                # Add depot return
                schedule_rows.append({
                    'vehicle_id': vehicle_id,
                    'stop_num': len(schedule['trips']) + 2,
                    'type': 'depot',
                    'id': depot_id,
                    'location': depot_id
                })
                
                vehicle_id += 1
        
        schedules_df = pd.DataFrame(schedule_rows)
        return schedules_df
    else:
        print("No optimal solution found.")
        return None

def analyze_lp_relaxation_set_partitioning(trips_df, connections_df, depot_id='fellsway_garage', max_vehicles=10):
    """
    Analyze the LP relaxation of set partitioning formulation using Gurobi
    
    Args:
        trips_df: DataFrame of trip nodes
        connections_df: DataFrame of connections between trips
        depot_id: ID of the depot
        max_vehicles: Maximum number of vehicles available
    """
    print("Analyzing LP relaxation of set partitioning formulation with Gurobi...")
    start_time = time.time()
    
    # Create a list of all trips
    trips = trips_df['half_trip_id'].tolist()
    
    # Create a graph representation for the network
    graph = {}
    for _, conn in connections_df.iterrows():
        from_id = conn['from_id']
        to_id = conn['to_id']
        cost = conn['total_cost']
        
        if from_id not in graph:
            graph[from_id] = {}
        graph[from_id][to_id] = cost
    
    # Generate all feasible vehicle schedules
    feasible_schedules = []
    
    # Use a recursive function to find all paths from depot to depot
    def find_paths(current_path, visited_trips, current_cost):
        current_node = current_path[-1]
        
        # If we've returned to the depot and visited at least one trip, we have a feasible schedule
        if current_node == depot_id and len(current_path) > 2:
            # Extract just the trip IDs (exclude depot at start and end)
            trip_ids = [node for node in current_path[1:-1]]
            feasible_schedules.append({
                'trips': trip_ids,
                'cost': current_cost
            })
            return
        
        # If we're at the maximum path length, stop recursion
        if len(current_path) > len(trips) + 2:
            return
        
        # Try all possible next nodes
        if current_node in graph:
            for next_node, edge_cost in graph[current_node].items():
                # Skip if this trip has already been visited
                if next_node in trips and next_node in visited_trips:
                    continue
                
                # Add this node to the path
                new_path = current_path + [next_node]
                new_visited = visited_trips.copy()
                if next_node in trips:
                    new_visited.add(next_node)
                
                # Continue recursion
                find_paths(new_path, new_visited, current_cost + edge_cost)
    
    # Start with paths from the depot
    find_paths([depot_id], set(), 0)
    
    print(f"Generated {len(feasible_schedules)} feasible schedules")
    
    # Create the Gurobi model
    model = gp.Model("MDVSP_SetPartitioning_LP")
    
    # Create continuous variables (for LP relaxation)
    y = {}
    for i, schedule in enumerate(feasible_schedules):
        y[i] = model.addVar(lb=0, ub=1, vtype=GRB.CONTINUOUS, name=f"y_{i}")
    
    # Update model to incorporate new variables
    model.update()
    
    # Objective function: minimize total cost
    obj = gp.LinExpr()
    for i, schedule in enumerate(feasible_schedules):
        obj += y[i] * schedule['cost']
    
    model.setObjective(obj, GRB.MINIMIZE)
    
    # Constraint 1: Each trip must be covered exactly once
    for trip in trips:
        constr = gp.LinExpr()
        for i, schedule in enumerate(feasible_schedules):
            if trip in schedule['trips']:
                constr += y[i]
        model.addConstr(constr == 1, f"Trip_{trip}_must_be_served")
    
    # Constraint 2: Maximum number of vehicles
    vehicles_constr = gp.LinExpr()
    for i in range(len(feasible_schedules)):
        vehicles_constr += y[i]
    model.addConstr(vehicles_constr <= max_vehicles, "Max_vehicles")
    
    # Set solver parameters
    model.setParam('OutputFlag', 0)  # Suppress solver output
    
    # Solve the LP relaxation
    model.optimize()
    
    end_time = time.time()
    solution_time = end_time - start_time
    
    print(f"Set partitioning LP relaxation - Status: {model.status}")
    if model.status == GRB.OPTIMAL:
        print(f"Set partitioning LP relaxation - Objective: {model.objVal}")
    print(f"Solution time: {solution_time:.2f} seconds")

def analyze_lp_relaxation(trips_df, connections_df, depot_id=DEPOT_ID, max_vehicles=10):
    """
    Analyze the LP relaxation of arc-flow formulation using Gurobi
    
    Args:
        trips_df: DataFrame of trip nodes
        connections_df: DataFrame of connections between trips
        depot_id: ID of the depot
        max_vehicles: Maximum number of vehicles available
    """
    print("Analyzing LP relaxation of arc-flow formulation with Gurobi...")
    start_time = time.time()
    
    # Create a list of all trips
    trips = trips_df['half_trip_id'].tolist()
    
    # Create a dictionary to store connections
    connections = {}
    for _, conn in connections_df.iterrows():
        from_id = conn['from_id']
        to_id = conn['to_id']
        total_cost = conn['total_cost']
        
        # Add connection to the dictionary
        if from_id not in connections:
            connections[from_id] = {}
        connections[from_id][to_id] = total_cost
    
    # Create the Gurobi model for arc-flow
    model = gp.Model("MDVSP_ArcFlow_LP")
    
    # Create continuous variables
    x = {}
    for _, conn in connections_df.iterrows():
        from_id = conn['from_id']
        to_id = conn['to_id']
        
        
        # Create continuous variable for this connection
        x[(from_id, to_id)] = model.addVar(lb=0, ub=1, vtype=GRB.CONTINUOUS, name=f"x_{from_id}_{to_id}")
    
    # Update model to incorporate new variables
    model.update()
    
    # Objective function: minimize total cost
    obj = gp.LinExpr()
    for _, conn in connections_df.iterrows():
        from_id = conn['from_id']
        to_id = conn['to_id']
        obj += x[(from_id, to_id)] * conn['total_cost']
    
    model.setObjective(obj, GRB.MINIMIZE)
    
    # Constraint 1: Flow conservation for trips
    for trip in trips:
        # Inflow equals outflow for each trip
        inflow = gp.LinExpr()
        outflow = gp.LinExpr()
        
        for from_id in connections:
            if trip in connections[from_id]:
                inflow += x[(from_id, trip)]
        
        if trip in connections:
            for to_id in connections[trip]:
                outflow += x[(trip, to_id)]
        
        model.addConstr(inflow == 1, f"Trip_{trip}_must_be_served")
        model.addConstr(inflow == outflow, f"Flow_conservation_{trip}")
    
    # Constraint 2: Maximum number of vehicles
    outflow_from_depot = gp.LinExpr()
    if depot_id in connections:
        for to_id in connections[depot_id]:
            outflow_from_depot += x[(depot_id, to_id)]
    
    model.addConstr(outflow_from_depot <= max_vehicles, "Max_vehicles")
    
    # Set solver parameters
    model.setParam('OutputFlag', 0)  # Suppress solver output
    
    # Solve the LP relaxation
    model.optimize()
    
    end_time = time.time()
    solution_time = end_time - start_time
    
    print(f"Arc-flow LP relaxation - Status: {model.status}")
    if model.status == GRB.OPTIMAL:
        print(f"Arc-flow LP relaxation - Objective: {model.objVal}")
    print(f"Solution time: {solution_time:.2f} seconds")

def main():
    # Input and output paths
    input_csv = 'filtered_route100_4h_7h.csv'
    trip_nodes_output = 'route100_4h_7h_trip_nodes_gurobi.csv'
    connections_output = 'route100_4h_7h_connections_gurobi.csv'
    arc_flow_solution_output = 'route100_4h_7h_arc_flow_solution_gurobi.csv'
    set_partitioning_solution_output = 'route100_4h_7h_set_partitioning_solution_gurobi.csv'
    
    try:
        # Step 1: Create trip nodes
        print("\n--- STEP 1: Creating trip nodes ---")
        trips_df = create_trip_nodes(input_csv)
        
        # Print summary
        print(f"\nCreated {len(trips_df)} trip nodes")
        if not trips_df.empty:
            print("\nSample trip nodes:")
            print(trips_df.head())
            
            # Save to CSV
            trips_df.to_csv(trip_nodes_output, index=False)
            print(f"\nTrip nodes saved to {trip_nodes_output}")
        
        # Step 2: Calculate connections and costs
        print("\n--- STEP 2: Calculating connections and costs ---")
        connections_df = calculate_costs(trips_df)
        
        # Print summary
        print(f"\nCreated {len(connections_df)} connections")
        print("\nSample connections:")
        print(connections_df.head())
        
        # Save to CSV
        connections_df.to_csv(connections_output, index=False)
        print(f"\nConnections saved to {connections_output}")
        
        # Step 3: Analyze LP relaxation of arc-flow formulation
        print("\n--- STEP 3: Analyzing LP relaxation of arc-flow formulation ---")
        analyze_lp_relaxation(trips_df, connections_df)
        
        # Step 4: Analyze LP relaxation of set partitioning formulation
        print("\n--- STEP 4: Analyzing LP relaxation of set partitioning formulation ---")
        analyze_lp_relaxation_set_partitioning(trips_df, connections_df)
        
        # Step 5: Solve using arc-flow formulation
        print("\n--- STEP 5: Solving with arc-flow formulation ---")
        arc_flow_solution = solve_arc_flow_mdvsp(trips_df, connections_df)
        
        if arc_flow_solution is not None:
            print("\nSample arc-flow solution:")
            print(arc_flow_solution.head())
            
            # Save to CSV
            arc_flow_solution.to_csv(arc_flow_solution_output, index=False)
            print(f"\nArc-flow solution saved to {arc_flow_solution_output}")
        
        # Step 6: Solve using set partitioning formulation
        print("\n--- STEP 6: Solving with set partitioning formulation ---")
        set_partitioning_solution = solve_set_partitioning_mdvsp(trips_df, connections_df)
        
        if set_partitioning_solution is not None:
            print("\nSample set partitioning solution:")
            print(set_partitioning_solution.head())
            
            # Save to CSV
            set_partitioning_solution.to_csv(set_partitioning_solution_output, index=False)
            print(f"\nSet partitioning solution saved to {set_partitioning_solution_output}")
    
    except Exception as e:
        print(f"Error: {e}")

if __name__ == "__main__":
    main()


--- STEP 1: Creating trip nodes ---
Total records: 25
Unique half_trip_ids: 13
Unique stop_ids: [52711  8301  8302 52720]
Unique time_point_ids: ['welst' 'elm']

Created 12 trip nodes

Sample trip nodes:
   half_trip_id  route_id direction_id start_location end_location  \
0      65802629       100     Outbound          welst          elm   
1      65802630       100      Inbound            elm        welst   
2      65802631       100     Outbound          welst          elm   
3      65802632       100      Inbound            elm        welst   
4      65802633       100     Outbound          welst          elm   

  scheduled_start_time  scheduled_end_time   actual_start_time  \
0  2025-04-29 16:00:00 2025-04-29 16:11:00 2025-04-29 16:06:48   
1  2025-04-29 16:15:00 2025-04-29 16:27:00 2025-04-29 16:23:13   
2  2025-04-29 16:30:00 2025-04-29 16:41:00 2025-04-29 16:38:27   
3  2025-04-29 16:45:00 2025-04-29 16:57:00 2025-04-29 16:52:23   
4  2025-04-29 17:00:00 2025-04-29 17:11:00 2

/var/folders/v8/1793d44577q9lqgx_01s8skm0000gn/T/ipykernel_21153/2576639542.py:88: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  trips_df[col] = pd.to_datetime(trips_df[col], errors='coerce')
/var/folders/v8/1793d44577q9lqgx_01s8skm0000gn/T/ipykernel_21153/2576639542.py:88: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  trips_df[col] = pd.to_datetime(trips_df[col], errors='coerce')
/var/folders/v8/1793d44577q9lqgx_01s8skm0000gn/T/ipykernel_21153/2576639542.py:88: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  trips_df[col] = pd.to_datetime(trips_df[col], errors='coerce')
/var

## b. 4 hours

In [6]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import gurobipy as gp
from gurobipy import GRB
import time

# Define bus operation parameters
BUS_PARAMS = {
    'avg_speed_mph': 12,  # Average bus speed in mph
    'cost_per_mile': 2.5,  # Operating cost per mile
    'cost_per_minute': 1.0,  # Cost per minute of operation
    'waiting_cost_factor': 0.5,  # Cost factor for waiting time
    'delay_risk_factor': 2.0  # Cost factor for delay risk
}

# Define the depot
DEPOT_ID = 'fellsway_garage'

# Define distances from Google Maps screenshots (in miles)
DISTANCES = {
    'depot_to_welst': 2.0,  # Fellsway Garage to Wellington St
    'depot_to_elmst': 1.4,  # Fellsway Garage to Elm St
    'welst_to_depot': 2.0,  # Wellington St to Fellsway Garage
    'elmst_to_depot': 1.4,  # Elm St to Fellsway Garage
    'elmst_to_welst': 1.5,  # Estimated distance between stops
    'welst_to_elmst': 1.5   # Estimated distance between stops
}

def calculate_travel_time(distance, speed=BUS_PARAMS['avg_speed_mph']):
    """Calculate travel time based on distance and speed"""
    return (distance / speed) * 60

def create_trip_nodes(csv_path):
    """Create trip nodes from the filtered CSV data"""
    # Read the filtered data
    df = pd.read_csv(csv_path)
    
    # Print data summary
    print(f"Total records: {len(df)}")
    print(f"Unique half_trip_ids: {df['half_trip_id'].nunique()}")
    print(f"Unique stop_ids: {df['stop_id'].unique()}")
    print(f"Unique time_point_ids: {df['time_point_id'].unique()}")
    
    # Map time_point_ids to our location codes
    location_map = {
        'elmst': 'elmst',
        'welst': 'welst'
    }
    
    # Group by half_trip_id
    trips = []
    
    # For each half_trip_id, find its startpoint and endpoint
    for half_trip_id, group in df.groupby('half_trip_id'):
        startpoints = group[group['point_type'] == 'Startpoint']
        endpoints = group[group['point_type'] == 'Endpoint']
        
        if not startpoints.empty and not endpoints.empty:
            # Get start and end locations
            start_time_point = startpoints['time_point_id'].iloc[0]
            end_time_point = endpoints['time_point_id'].iloc[0]
            
            # Map to our location codes
            start_location = location_map.get(start_time_point, start_time_point)
            end_location = location_map.get(end_time_point, end_time_point)
            
            trip = {
                'half_trip_id': half_trip_id,
                'route_id': group['route_id'].iloc[0],
                'direction_id': group['direction_id'].iloc[0],
                'start_location': start_location,
                'end_location': end_location,
                'scheduled_start_time': startpoints['scheduled'].iloc[0],
                'scheduled_end_time': endpoints['scheduled'].iloc[0],
                'actual_start_time': startpoints['actual'].iloc[0],
                'actual_end_time': endpoints['actual'].iloc[0]
            }
            trips.append(trip)
    
    # Convert to DataFrame
    trips_df = pd.DataFrame(trips)
    
    # Parse datetime strings
    for col in ['scheduled_start_time', 'scheduled_end_time', 'actual_start_time', 'actual_end_time']:
        if col in trips_df.columns:
            # Convert times to datetime objects
            trips_df[col] = pd.to_datetime(trips_df[col], errors='coerce')
    
    return trips_df

def calculate_costs(trips_df, depot_id=DEPOT_ID):
    """
    Calculate costs between trips and depot connections using distances from Google Maps
    
    Args:
        trips_df: DataFrame of trip nodes
        depot_id: ID of the depot
        
    Returns:
        DataFrame with trip-to-trip costs and depot connections
    """
    # Get parameters
    cost_per_mile = BUS_PARAMS['cost_per_mile']
    cost_per_minute = BUS_PARAMS['cost_per_minute']
    waiting_cost_factor = BUS_PARAMS['waiting_cost_factor']
    delay_risk_factor = BUS_PARAMS['delay_risk_factor']
    
    # Create empty lists to store connections
    connections = []
    
    # Add depot to start connections
    for _, trip in trips_df.iterrows():
        # Get distance from depot to this trip's start location
        if trip['start_location'] == 'welst':
            distance = DISTANCES['depot_to_welst']
        elif trip['start_location'] == 'elmst':
            distance = DISTANCES['depot_to_elmst']
        else:
            # Use average distance for any other locations
            distance = (DISTANCES['depot_to_welst'] + DISTANCES['depot_to_elmst']) / 2
            
        # Calculate travel time
        travel_time = calculate_travel_time(distance)
        
        # Calculate costs
        deadheading_cost = distance * cost_per_mile + travel_time * cost_per_minute
        
        connection = {
            'from_id': depot_id,
            'to_id': trip['half_trip_id'],
            'from_type': 'depot',
            'to_type': 'trip',
            'from_location': depot_id,
            'to_location': trip['start_location'],
            'distance_miles': distance,
            'travel_time_minutes': travel_time,
            'deadheading_cost': deadheading_cost,
            'waiting_cost': 0,
            'delay_risk_cost': 0,
            'total_cost': deadheading_cost
        }
        connections.append(connection)
    
    # Add end to depot connections
    for _, trip in trips_df.iterrows():
        # Get distance from this trip's end location to depot
        if trip['end_location'] == 'welst':
            distance = DISTANCES['welst_to_depot']
        elif trip['end_location'] == 'elmst':
            distance = DISTANCES['elmst_to_depot']
        else:
            # Use average distance for any other locations
            distance = (DISTANCES['welst_to_depot'] + DISTANCES['elmst_to_depot']) / 2
            
        # Calculate travel time
        travel_time = calculate_travel_time(distance)
        
        # Calculate costs
        deadheading_cost = distance * cost_per_mile + travel_time * cost_per_minute
        
        connection = {
            'from_id': trip['half_trip_id'],
            'to_id': depot_id,
            'from_type': 'trip',
            'to_type': 'depot',
            'from_location': trip['end_location'],
            'to_location': depot_id,
            'distance_miles': distance,
            'travel_time_minutes': travel_time,
            'deadheading_cost': deadheading_cost,
            'waiting_cost': 0,
            'delay_risk_cost': 0,
            'total_cost': deadheading_cost
        }
        connections.append(connection)
    
    # Add trip-to-trip connections
    for i, trip_i in trips_df.iterrows():
        for j, trip_j in trips_df.iterrows():
            # Skip same trip
            if i == j:
                continue
                
            # Check if trip_j can follow trip_i (time compatibility)
            if pd.isnull(trip_i['scheduled_end_time']) or pd.isnull(trip_j['scheduled_start_time']):
                continue
            
            # Get distance between end of trip_i and start of trip_j
            from_loc = trip_i['end_location']
            to_loc = trip_j['start_location']
            
            if from_loc == 'elmst' and to_loc == 'welst':
                distance = DISTANCES['elmst_to_welst']
            elif from_loc == 'welst' and to_loc == 'elmst':
                distance = DISTANCES['welst_to_elmst']
            elif from_loc == to_loc:
                distance = 0.1  # Short distance if same location
            else:
                # Use average for any other combinations
                distance = (DISTANCES['elmst_to_welst'] + DISTANCES['welst_to_elmst']) / 2
            
            # Calculate travel time
            travel_time = calculate_travel_time(distance)
            
            # Calculate earliest possible arrival time at start of trip_j
            earliest_arrival = trip_i['scheduled_end_time'] + timedelta(minutes=travel_time)
            
            # Check if trip_j can be served after trip_i
            if earliest_arrival <= trip_j['scheduled_start_time']:
                # Calculate waiting time
                waiting_time = (trip_j['scheduled_start_time'] - earliest_arrival).total_seconds() / 60
                
                # Calculate delay risk (if trip_i has history of delays)
                if not pd.isnull(trip_i['actual_end_time']) and not pd.isnull(trip_i['scheduled_end_time']):
                    delay = (trip_i['actual_end_time'] - trip_i['scheduled_end_time']).total_seconds() / 60
                    delay_risk = max(0, delay)
                else:
                    delay_risk = 0
                
                # Calculate costs
                deadheading_cost = distance * cost_per_mile + travel_time * cost_per_minute
                waiting_cost = waiting_time * waiting_cost_factor * cost_per_minute
                delay_risk_cost = delay_risk * delay_risk_factor * cost_per_minute
                total_cost = deadheading_cost + waiting_cost + delay_risk_cost
                
                connection = {
                    'from_id': trip_i['half_trip_id'],
                    'to_id': trip_j['half_trip_id'],
                    'from_type': 'trip',
                    'to_type': 'trip',
                    'from_location': trip_i['end_location'],
                    'to_location': trip_j['start_location'],
                    'distance_miles': distance,
                    'travel_time_minutes': travel_time,
                    'waiting_time_minutes': waiting_time,
                    'deadheading_cost': deadheading_cost,
                    'waiting_cost': waiting_cost,
                    'delay_risk_cost': delay_risk_cost,
                    'total_cost': total_cost
                }
                connections.append(connection)
    
    # Convert to DataFrame
    connections_df = pd.DataFrame(connections)
    return connections_df

def solve_arc_flow_mdvsp(trips_df, connections_df, depot_id=DEPOT_ID, max_vehicles=10):
    """
    Solve the MDVSP using the arc-flow formulation with Gurobi
    
    Args:
        trips_df: DataFrame of trip nodes
        connections_df: DataFrame of connections between trips
        depot_id: ID of the depot
        max_vehicles: Maximum number of vehicles available
        
    Returns:
        Solution DataFrame with vehicle schedules
    """
    print("Solving MDVSP using arc-flow formulation with Gurobi...")
    start_time = time.time()
    
    # Create a list of all trips
    trips = trips_df['half_trip_id'].tolist()
    
    # Create a dictionary to store connections
    connections = {}
    for _, conn in connections_df.iterrows():
        from_id = conn['from_id']
        to_id = conn['to_id']
        total_cost = conn['total_cost']
        
        # Add connection to the dictionary
        if from_id not in connections:
            connections[from_id] = {}
        connections[from_id][to_id] = total_cost
    
    # Create the Gurobi model
    model = gp.Model("MDVSP_ArcFlow")
    
    # Create variables
    x = {}
    for _, conn in connections_df.iterrows():
        from_id = conn['from_id']
        to_id = conn['to_id']
        
        # Create binary variable for this connection
        x[(from_id, to_id)] = model.addVar(vtype=GRB.BINARY, name=f"x_{from_id}_{to_id}")
    
    # Update the model to include new variables
    model.update()
    
    # Objective function: minimize total cost
    obj = gp.LinExpr()
    for _, conn in connections_df.iterrows():
        from_id = conn['from_id']
        to_id = conn['to_id']
        obj += x[(from_id, to_id)] * conn['total_cost']
    
    model.setObjective(obj, GRB.MINIMIZE)
    
    # Constraint 1: Flow conservation for trips
    for trip in trips:
        # Inflow equals outflow for each trip
        inflow = gp.LinExpr()
        outflow = gp.LinExpr()
        
        for from_id in connections:
            if trip in connections[from_id]:
                inflow += x[(from_id, trip)]
        
        if trip in connections:
            for to_id in connections[trip]:
                outflow += x[(trip, to_id)]
        
        model.addConstr(inflow == 1, f"Trip_{trip}_must_be_served")
        model.addConstr(inflow == outflow, f"Flow_conservation_{trip}")
    
    # Constraint 2: Maximum number of vehicles
    outflow_from_depot = gp.LinExpr()
    if depot_id in connections:
        for to_id in connections[depot_id]:
            outflow_from_depot += x[(depot_id, to_id)]
    
    model.addConstr(outflow_from_depot <= max_vehicles, "Max_vehicles")
    
    # Set time limit and solver parameters
    model.setParam('TimeLimit', 300)  # 5 minutes time limit
    model.setParam('OutputFlag', 1)   # Display solver output
    
    # Solve the model
    model.optimize()
    
    end_time = time.time()
    solution_time = end_time - start_time
    
    print(f"Status: {model.status}")
    if model.status == GRB.OPTIMAL or model.status == GRB.TIME_LIMIT:
        print(f"Total cost: {model.objVal}")
    print(f"Solution time: {solution_time:.2f} seconds")
    
    # Extract solution
    if model.status == GRB.OPTIMAL or model.status == GRB.TIME_LIMIT:
        # Create a list to store vehicle schedules
        vehicle_schedules = []
        
        # Start with connections from depot
        if depot_id in connections:
            for to_id in connections[depot_id]:
                if x[(depot_id, to_id)].x > 0.5:  # If this connection is used
                    # Start a new vehicle schedule
                    schedule = [{'type': 'depot', 'id': depot_id, 'location': depot_id}]
                    
                    # Follow the path of this vehicle
                    current_id = to_id
                    while current_id != depot_id:
                        # Add this trip to the schedule
                        trip_info = trips_df[trips_df['half_trip_id'] == current_id].iloc[0].to_dict()
                        schedule.append({
                            'type': 'trip',
                            'id': current_id,
                            'start_location': trip_info['start_location'],
                            'end_location': trip_info['end_location'],
                            'start_time': trip_info['scheduled_start_time'],
                            'end_time': trip_info['scheduled_end_time'],
                            'route_id': trip_info['route_id'],
                            'direction_id': trip_info['direction_id']
                        })
                        
                        # Find the next trip/depot
                        next_id = None
                        if current_id in connections:
                            for possible_next in connections[current_id]:
                                if x[(current_id, possible_next)].x > 0.5:
                                    next_id = possible_next
                                    break
                        
                        if next_id is None:
                            print(f"Warning: No next trip found for {current_id}")
                            break
                            
                        current_id = next_id
                    
                    # Close the schedule with return to depot
                    schedule.append({'type': 'depot', 'id': depot_id, 'location': depot_id})
                    
                    # Add this schedule to the list
                    vehicle_schedules.append(schedule)
        
        # Convert to DataFrame for easier analysis
        schedule_rows = []
        for vehicle_id, schedule in enumerate(vehicle_schedules):
            for stop_num, stop in enumerate(schedule):
                row = {
                    'vehicle_id': vehicle_id + 1,
                    'stop_num': stop_num + 1,
                    'type': stop['type'],
                    'id': stop['id']
                }
                
                # Add location information
                if 'location' in stop:
                    row['location'] = stop['location']
                
                # Add trip details if this is a trip
                if stop['type'] == 'trip':
                    row.update({
                        'start_location': stop['start_location'],
                        'end_location': stop['end_location'],
                        'start_time': stop['start_time'],
                        'end_time': stop['end_time'],
                        'route_id': stop['route_id'],
                        'direction_id': stop['direction_id']
                    })
                
                schedule_rows.append(row)
        
        schedules_df = pd.DataFrame(schedule_rows)
        return schedules_df
    else:
        print("No optimal solution found.")
        return None

def solve_set_partitioning_mdvsp(trips_df, connections_df, depot_id='fellsway_garage', max_vehicles=10):
    """
    Solve the MDVSP using the set partitioning formulation with Gurobi
    
    Args:
        trips_df: DataFrame of trip nodes
        connections_df: DataFrame of connections between trips
        depot_id: ID of the depot
        max_vehicles: Maximum number of vehicles available
        
    Returns:
        Solution DataFrame with vehicle schedules
    """
    print("Solving MDVSP using set partitioning formulation with Gurobi...")
    start_time = time.time()
    
    # Create a list of all trips
    trips = trips_df['half_trip_id'].tolist()
    
    # Create a graph representation for the network
    graph = {}
    for _, conn in connections_df.iterrows():
        from_id = conn['from_id']
        to_id = conn['to_id']
        cost = conn['total_cost']
        
        if from_id not in graph:
            graph[from_id] = {}
        graph[from_id][to_id] = cost
    
    # Generate all feasible vehicle schedules (paths from depot to depot)
    print("Generating feasible vehicle schedules...")
    feasible_schedules = []
    
    # Use a recursive function to find all paths from depot to depot
    def find_paths(current_path, visited_trips, current_cost):
        current_node = current_path[-1]
        
        # If we've returned to the depot and visited at least one trip, we have a feasible schedule
        if current_node == depot_id and len(current_path) > 2:
            # Extract just the trip IDs (exclude depot at start and end)
            trip_ids = [node for node in current_path[1:-1]]
            feasible_schedules.append({
                'trips': trip_ids,
                'cost': current_cost,
                'path': current_path.copy()  # Keep full path for reference
            })
            return
        
        # If we're at the maximum path length, stop recursion
        if len(current_path) > len(trips) + 2:
            return
        
        # Try all possible next nodes
        if current_node in graph:
            for next_node, edge_cost in graph[current_node].items():
                # Skip if this trip has already been visited
                if next_node in trips and next_node in visited_trips:
                    continue
                
                # Add this node to the path
                new_path = current_path + [next_node]
                new_visited = visited_trips.copy()
                if next_node in trips:
                    new_visited.add(next_node)
                
                # Continue recursion
                find_paths(new_path, new_visited, current_cost + edge_cost)
    
    # Start with paths from the depot
    find_paths([depot_id], set(), 0)
    
    print(f"Generated {len(feasible_schedules)} feasible schedules")
    
    # Create the Gurobi model
    model = gp.Model("MDVSP_SetPartitioning")
    
    # Create variables - one for each feasible schedule
    y = {}
    for i, schedule in enumerate(feasible_schedules):
        y[i] = model.addVar(vtype=GRB.BINARY, name=f"y_{i}")
    
    # Update model to incorporate new variables
    model.update()
    
    # Objective function: minimize total cost
    obj = gp.LinExpr()
    for i, schedule in enumerate(feasible_schedules):
        obj += y[i] * schedule['cost']
    
    model.setObjective(obj, GRB.MINIMIZE)
    
    # Constraint 1: Each trip must be covered exactly once
    for trip in trips:
        constr = gp.LinExpr()
        for i, schedule in enumerate(feasible_schedules):
            if trip in schedule['trips']:
                constr += y[i]
        model.addConstr(constr == 1, f"Trip_{trip}_must_be_served")
    
    # Constraint 2: Maximum number of vehicles
    vehicles_constr = gp.LinExpr()
    for i in range(len(feasible_schedules)):
        vehicles_constr += y[i]
    model.addConstr(vehicles_constr <= max_vehicles, "Max_vehicles")
    
    # Set time limit and solver parameters
    model.setParam('TimeLimit', 300)  # 5 minutes time limit
    model.setParam('OutputFlag', 1)   # Display solver output
    
    # Solve the model
    model.optimize()
    
    end_time = time.time()
    solution_time = end_time - start_time
    
    print(f"Status: {model.status}")
    if model.status == GRB.OPTIMAL or model.status == GRB.TIME_LIMIT:
        print(f"Total cost: {model.objVal}")
    print(f"Solution time: {solution_time:.2f} seconds")
    
    # Extract solution
    if model.status == GRB.OPTIMAL or model.status == GRB.TIME_LIMIT:
        # Create a list to store vehicle schedules
        schedule_rows = []
        
        # Process each selected schedule
        vehicle_id = 1
        for i, schedule in enumerate(feasible_schedules):
            if y[i].x > 0.5:  # If this schedule is used
                path = schedule['path']
                
                # Add depot departure
                schedule_rows.append({
                    'vehicle_id': vehicle_id,
                    'stop_num': 1,
                    'type': 'depot',
                    'id': depot_id,
                    'location': depot_id
                })
                
                # Add each trip in the schedule
                for stop_num, trip_id in enumerate(schedule['trips']):
                    # Get trip details
                    trip_info = trips_df[trips_df['half_trip_id'] == trip_id].iloc[0].to_dict()
                    
                    schedule_rows.append({
                        'vehicle_id': vehicle_id,
                        'stop_num': stop_num + 2,
                        'type': 'trip',
                        'id': trip_id,
                        'start_location': trip_info['start_location'],
                        'end_location': trip_info['end_location'],
                        'start_time': trip_info['scheduled_start_time'],
                        'end_time': trip_info['scheduled_end_time'],
                        'route_id': trip_info['route_id'],
                        'direction_id': trip_info['direction_id']
                    })
                
                # Add depot return
                schedule_rows.append({
                    'vehicle_id': vehicle_id,
                    'stop_num': len(schedule['trips']) + 2,
                    'type': 'depot',
                    'id': depot_id,
                    'location': depot_id
                })
                
                vehicle_id += 1
        
        schedules_df = pd.DataFrame(schedule_rows)
        return schedules_df
    else:
        print("No optimal solution found.")
        return None

def analyze_lp_relaxation_set_partitioning(trips_df, connections_df, depot_id='fellsway_garage', max_vehicles=10):
    """
    Analyze the LP relaxation of set partitioning formulation using Gurobi
    
    Args:
        trips_df: DataFrame of trip nodes
        connections_df: DataFrame of connections between trips
        depot_id: ID of the depot
        max_vehicles: Maximum number of vehicles available
    """
    print("Analyzing LP relaxation of set partitioning formulation with Gurobi...")
    start_time = time.time()
    
    # Create a list of all trips
    trips = trips_df['half_trip_id'].tolist()
    
    # Create a graph representation for the network
    graph = {}
    for _, conn in connections_df.iterrows():
        from_id = conn['from_id']
        to_id = conn['to_id']
        cost = conn['total_cost']
        
        if from_id not in graph:
            graph[from_id] = {}
        graph[from_id][to_id] = cost
    
    # Generate all feasible vehicle schedules
    feasible_schedules = []
    
    # Use a recursive function to find all paths from depot to depot
    def find_paths(current_path, visited_trips, current_cost):
        current_node = current_path[-1]
        
        # If we've returned to the depot and visited at least one trip, we have a feasible schedule
        if current_node == depot_id and len(current_path) > 2:
            # Extract just the trip IDs (exclude depot at start and end)
            trip_ids = [node for node in current_path[1:-1]]
            feasible_schedules.append({
                'trips': trip_ids,
                'cost': current_cost
            })
            return
        
        # If we're at the maximum path length, stop recursion
        if len(current_path) > len(trips) + 2:
            return
        
        # Try all possible next nodes
        if current_node in graph:
            for next_node, edge_cost in graph[current_node].items():
                # Skip if this trip has already been visited
                if next_node in trips and next_node in visited_trips:
                    continue
                
                # Add this node to the path
                new_path = current_path + [next_node]
                new_visited = visited_trips.copy()
                if next_node in trips:
                    new_visited.add(next_node)
                
                # Continue recursion
                find_paths(new_path, new_visited, current_cost + edge_cost)
    
    # Start with paths from the depot
    find_paths([depot_id], set(), 0)
    
    print(f"Generated {len(feasible_schedules)} feasible schedules")
    
    # Create the Gurobi model
    model = gp.Model("MDVSP_SetPartitioning_LP")
    
    # Create continuous variables (for LP relaxation)
    y = {}
    for i, schedule in enumerate(feasible_schedules):
        y[i] = model.addVar(lb=0, ub=1, vtype=GRB.CONTINUOUS, name=f"y_{i}")
    
    # Update model to incorporate new variables
    model.update()
    
    # Objective function: minimize total cost
    obj = gp.LinExpr()
    for i, schedule in enumerate(feasible_schedules):
        obj += y[i] * schedule['cost']
    
    model.setObjective(obj, GRB.MINIMIZE)
    
    # Constraint 1: Each trip must be covered exactly once
    for trip in trips:
        constr = gp.LinExpr()
        for i, schedule in enumerate(feasible_schedules):
            if trip in schedule['trips']:
                constr += y[i]
        model.addConstr(constr == 1, f"Trip_{trip}_must_be_served")
    
    # Constraint 2: Maximum number of vehicles
    vehicles_constr = gp.LinExpr()
    for i in range(len(feasible_schedules)):
        vehicles_constr += y[i]
    model.addConstr(vehicles_constr <= max_vehicles, "Max_vehicles")
    
    # Set solver parameters
    model.setParam('OutputFlag', 0)  # Suppress solver output
    
    # Solve the LP relaxation
    model.optimize()
    
    end_time = time.time()
    solution_time = end_time - start_time
    
    print(f"Set partitioning LP relaxation - Status: {model.status}")
    if model.status == GRB.OPTIMAL:
        print(f"Set partitioning LP relaxation - Objective: {model.objVal}")
    print(f"Solution time: {solution_time:.2f} seconds")

def analyze_lp_relaxation(trips_df, connections_df, depot_id=DEPOT_ID, max_vehicles=10):
    """
    Analyze the LP relaxation of arc-flow formulation using Gurobi
    
    Args:
        trips_df: DataFrame of trip nodes
        connections_df: DataFrame of connections between trips
        depot_id: ID of the depot
        max_vehicles: Maximum number of vehicles available
    """
    print("Analyzing LP relaxation of arc-flow formulation with Gurobi...")
    start_time = time.time()
    
    # Create a list of all trips
    trips = trips_df['half_trip_id'].tolist()
    
    # Create a dictionary to store connections
    connections = {}
    for _, conn in connections_df.iterrows():
        from_id = conn['from_id']
        to_id = conn['to_id']
        total_cost = conn['total_cost']
        
        # Add connection to the dictionary
        if from_id not in connections:
            connections[from_id] = {}
        connections[from_id][to_id] = total_cost
    
    # Create the Gurobi model for arc-flow
    model = gp.Model("MDVSP_ArcFlow_LP")
    
    # Create continuous variables
    x = {}
    for _, conn in connections_df.iterrows():
        from_id = conn['from_id']
        to_id = conn['to_id']
        
        
        # Create continuous variable for this connection
        x[(from_id, to_id)] = model.addVar(lb=0, ub=1, vtype=GRB.CONTINUOUS, name=f"x_{from_id}_{to_id}")
    
    # Update model to incorporate new variables
    model.update()
    
    # Objective function: minimize total cost
    obj = gp.LinExpr()
    for _, conn in connections_df.iterrows():
        from_id = conn['from_id']
        to_id = conn['to_id']
        obj += x[(from_id, to_id)] * conn['total_cost']
    
    model.setObjective(obj, GRB.MINIMIZE)
    
    # Constraint 1: Flow conservation for trips
    for trip in trips:
        # Inflow equals outflow for each trip
        inflow = gp.LinExpr()
        outflow = gp.LinExpr()
        
        for from_id in connections:
            if trip in connections[from_id]:
                inflow += x[(from_id, trip)]
        
        if trip in connections:
            for to_id in connections[trip]:
                outflow += x[(trip, to_id)]
        
        model.addConstr(inflow == 1, f"Trip_{trip}_must_be_served")
        model.addConstr(inflow == outflow, f"Flow_conservation_{trip}")
    
    # Constraint 2: Maximum number of vehicles
    outflow_from_depot = gp.LinExpr()
    if depot_id in connections:
        for to_id in connections[depot_id]:
            outflow_from_depot += x[(depot_id, to_id)]
    
    model.addConstr(outflow_from_depot <= max_vehicles, "Max_vehicles")
    
    # Set solver parameters
    model.setParam('OutputFlag', 0)  # Suppress solver output
    
    # Solve the LP relaxation
    model.optimize()
    
    end_time = time.time()
    solution_time = end_time - start_time
    
    print(f"Arc-flow LP relaxation - Status: {model.status}")
    if model.status == GRB.OPTIMAL:
        print(f"Arc-flow LP relaxation - Objective: {model.objVal}")
    print(f"Solution time: {solution_time:.2f} seconds")

def main():
    # Input and output paths
    input_csv = 'filtered_route100_16h_20h.csv'
    trip_nodes_output = 'route100_16h_20h_trip_nodes_gurobi.csv'
    connections_output = 'route100_16h_20h_connections_gurobi.csv'
    arc_flow_solution_output = 'route100_16h_20h_arc_flow_solution_gurobi.csv'
    set_partitioning_solution_output = 'route100_16h_20h_set_partitioning_solution_gurobi.csv'
    
    try:
        # Step 1: Create trip nodes
        print("\n--- STEP 1: Creating trip nodes ---")
        trips_df = create_trip_nodes(input_csv)
        
        # Print summary
        print(f"\nCreated {len(trips_df)} trip nodes")
        if not trips_df.empty:
            print("\nSample trip nodes:")
            print(trips_df.head())
            
            # Save to CSV
            trips_df.to_csv(trip_nodes_output, index=False)
            print(f"\nTrip nodes saved to {trip_nodes_output}")
        
        # Step 2: Calculate connections and costs
        print("\n--- STEP 2: Calculating connections and costs ---")
        connections_df = calculate_costs(trips_df)
        
        # Print summary
        print(f"\nCreated {len(connections_df)} connections")
        print("\nSample connections:")
        print(connections_df.head())
        
        # Save to CSV
        connections_df.to_csv(connections_output, index=False)
        print(f"\nConnections saved to {connections_output}")
        
        # Step 3: Analyze LP relaxation of arc-flow formulation
        print("\n--- STEP 3: Analyzing LP relaxation of arc-flow formulation ---")
        analyze_lp_relaxation(trips_df, connections_df)
        
        # Step 4: Analyze LP relaxation of set partitioning formulation
        print("\n--- STEP 4: Analyzing LP relaxation of set partitioning formulation ---")
        analyze_lp_relaxation_set_partitioning(trips_df, connections_df)
        
        # Step 5: Solve using arc-flow formulation
        print("\n--- STEP 5: Solving with arc-flow formulation ---")
        arc_flow_solution = solve_arc_flow_mdvsp(trips_df, connections_df)
        
        if arc_flow_solution is not None:
            print("\nSample arc-flow solution:")
            print(arc_flow_solution.head())
            
            # Save to CSV
            arc_flow_solution.to_csv(arc_flow_solution_output, index=False)
            print(f"\nArc-flow solution saved to {arc_flow_solution_output}")
        
        # Step 6: Solve using set partitioning formulation
        print("\n--- STEP 6: Solving with set partitioning formulation ---")
        set_partitioning_solution = solve_set_partitioning_mdvsp(trips_df, connections_df)
        
        if set_partitioning_solution is not None:
            print("\nSample set partitioning solution:")
            print(set_partitioning_solution.head())
            
            # Save to CSV
            set_partitioning_solution.to_csv(set_partitioning_solution_output, index=False)
            print(f"\nSet partitioning solution saved to {set_partitioning_solution_output}")
    
    except Exception as e:
        print(f"Error: {e}")

if __name__ == "__main__":
    main()


--- STEP 1: Creating trip nodes ---
Total records: 40
Unique half_trip_ids: 20
Unique stop_ids: [52711  8301  8302 52720]
Unique time_point_ids: ['welst' 'elm']


/var/folders/v8/1793d44577q9lqgx_01s8skm0000gn/T/ipykernel_21153/16876758.py:88: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  trips_df[col] = pd.to_datetime(trips_df[col], errors='coerce')
/var/folders/v8/1793d44577q9lqgx_01s8skm0000gn/T/ipykernel_21153/16876758.py:88: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  trips_df[col] = pd.to_datetime(trips_df[col], errors='coerce')
/var/folders/v8/1793d44577q9lqgx_01s8skm0000gn/T/ipykernel_21153/16876758.py:88: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  trips_df[col] = pd.to_datetime(trips_df[col], errors='coerce')
/var/folde


Created 20 trip nodes

Sample trip nodes:
   half_trip_id  route_id direction_id start_location end_location  \
0    65802629.0       100     Outbound          welst          elm   
1    65802630.0       100      Inbound            elm        welst   
2    65802631.0       100     Outbound          welst          elm   
3    65802632.0       100      Inbound            elm        welst   
4    65802633.0       100     Outbound          welst          elm   

  scheduled_start_time  scheduled_end_time   actual_start_time  \
0  2025-04-29 16:00:00 2025-04-29 16:11:00 2025-04-29 16:06:48   
1  2025-04-29 16:15:00 2025-04-29 16:27:00 2025-04-29 16:23:13   
2  2025-04-29 16:30:00 2025-04-29 16:41:00 2025-04-29 16:38:27   
3  2025-04-29 16:45:00 2025-04-29 16:57:00 2025-04-29 16:52:23   
4  2025-04-29 17:00:00 2025-04-29 17:11:00 2025-04-29 17:06:23   

      actual_end_time  
0 2025-04-29 16:18:58  
1 2025-04-29 16:36:13  
2 2025-04-29 16:47:10  
3 2025-04-29 17:04:50  
4 2025-04-29 17:18:

## c. 5 hours

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import gurobipy as gp
from gurobipy import GRB
import time

# Define bus operation parameters
BUS_PARAMS = {
    'avg_speed_mph': 12,  # Average bus speed in mph
    'cost_per_mile': 2.5,  # Operating cost per mile
    'cost_per_minute': 1.0,  # Cost per minute of operation
    'waiting_cost_factor': 0.5,  # Cost factor for waiting time
    'delay_risk_factor': 2.0  # Cost factor for delay risk
}

# Define the depot
DEPOT_ID = 'fellsway_garage'

# Define distances from Google Maps screenshots (in miles)
DISTANCES = {
    'depot_to_welst': 2.0,  # Fellsway Garage to Wellington St
    'depot_to_elmst': 1.4,  # Fellsway Garage to Elm St
    'welst_to_depot': 2.0,  # Wellington St to Fellsway Garage
    'elmst_to_depot': 1.4,  # Elm St to Fellsway Garage
    'elmst_to_welst': 1.5,  # Estimated distance between stops
    'welst_to_elmst': 1.5   # Estimated distance between stops
}

def calculate_travel_time(distance, speed=BUS_PARAMS['avg_speed_mph']):
    """Calculate travel time based on distance and speed"""
    return (distance / speed) * 60

def create_trip_nodes(csv_path):
    """Create trip nodes from the filtered CSV data"""
    # Read the filtered data
    df = pd.read_csv(csv_path)
    
    # Print data summary
    print(f"Total records: {len(df)}")
    print(f"Unique half_trip_ids: {df['half_trip_id'].nunique()}")
    print(f"Unique stop_ids: {df['stop_id'].unique()}")
    print(f"Unique time_point_ids: {df['time_point_id'].unique()}")
    
    # Map time_point_ids to our location codes
    location_map = {
        'elmst': 'elmst',
        'welst': 'welst'
    }
    
    # Group by half_trip_id
    trips = []
    
    # For each half_trip_id, find its startpoint and endpoint
    for half_trip_id, group in df.groupby('half_trip_id'):
        startpoints = group[group['point_type'] == 'Startpoint']
        endpoints = group[group['point_type'] == 'Endpoint']
        
        if not startpoints.empty and not endpoints.empty:
            # Get start and end locations
            start_time_point = startpoints['time_point_id'].iloc[0]
            end_time_point = endpoints['time_point_id'].iloc[0]
            
            # Map to our location codes
            start_location = location_map.get(start_time_point, start_time_point)
            end_location = location_map.get(end_time_point, end_time_point)
            
            trip = {
                'half_trip_id': half_trip_id,
                'route_id': group['route_id'].iloc[0],
                'direction_id': group['direction_id'].iloc[0],
                'start_location': start_location,
                'end_location': end_location,
                'scheduled_start_time': startpoints['scheduled'].iloc[0],
                'scheduled_end_time': endpoints['scheduled'].iloc[0],
                'actual_start_time': startpoints['actual'].iloc[0],
                'actual_end_time': endpoints['actual'].iloc[0]
            }
            trips.append(trip)
    
    # Convert to DataFrame
    trips_df = pd.DataFrame(trips)
    
    # Parse datetime strings
    for col in ['scheduled_start_time', 'scheduled_end_time', 'actual_start_time', 'actual_end_time']:
        if col in trips_df.columns:
            # Convert times to datetime objects
            trips_df[col] = pd.to_datetime(trips_df[col], errors='coerce')
    
    return trips_df

def calculate_costs(trips_df, depot_id=DEPOT_ID):
    """
    Calculate costs between trips and depot connections using distances from Google Maps
    
    Args:
        trips_df: DataFrame of trip nodes
        depot_id: ID of the depot
        
    Returns:
        DataFrame with trip-to-trip costs and depot connections
    """
    # Get parameters
    cost_per_mile = BUS_PARAMS['cost_per_mile']
    cost_per_minute = BUS_PARAMS['cost_per_minute']
    waiting_cost_factor = BUS_PARAMS['waiting_cost_factor']
    delay_risk_factor = BUS_PARAMS['delay_risk_factor']
    
    # Create empty lists to store connections
    connections = []
    
    # Add depot to start connections
    for _, trip in trips_df.iterrows():
        # Get distance from depot to this trip's start location
        if trip['start_location'] == 'welst':
            distance = DISTANCES['depot_to_welst']
        elif trip['start_location'] == 'elmst':
            distance = DISTANCES['depot_to_elmst']
        else:
            # Use average distance for any other locations
            distance = (DISTANCES['depot_to_welst'] + DISTANCES['depot_to_elmst']) / 2
            
        # Calculate travel time
        travel_time = calculate_travel_time(distance)
        
        # Calculate costs
        deadheading_cost = distance * cost_per_mile + travel_time * cost_per_minute
        
        connection = {
            'from_id': depot_id,
            'to_id': trip['half_trip_id'],
            'from_type': 'depot',
            'to_type': 'trip',
            'from_location': depot_id,
            'to_location': trip['start_location'],
            'distance_miles': distance,
            'travel_time_minutes': travel_time,
            'deadheading_cost': deadheading_cost,
            'waiting_cost': 0,
            'delay_risk_cost': 0,
            'total_cost': deadheading_cost
        }
        connections.append(connection)
    
    # Add end to depot connections
    for _, trip in trips_df.iterrows():
        # Get distance from this trip's end location to depot
        if trip['end_location'] == 'welst':
            distance = DISTANCES['welst_to_depot']
        elif trip['end_location'] == 'elmst':
            distance = DISTANCES['elmst_to_depot']
        else:
            # Use average distance for any other locations
            distance = (DISTANCES['welst_to_depot'] + DISTANCES['elmst_to_depot']) / 2
            
        # Calculate travel time
        travel_time = calculate_travel_time(distance)
        
        # Calculate costs
        deadheading_cost = distance * cost_per_mile + travel_time * cost_per_minute
        
        connection = {
            'from_id': trip['half_trip_id'],
            'to_id': depot_id,
            'from_type': 'trip',
            'to_type': 'depot',
            'from_location': trip['end_location'],
            'to_location': depot_id,
            'distance_miles': distance,
            'travel_time_minutes': travel_time,
            'deadheading_cost': deadheading_cost,
            'waiting_cost': 0,
            'delay_risk_cost': 0,
            'total_cost': deadheading_cost
        }
        connections.append(connection)
    
    # Add trip-to-trip connections
    for i, trip_i in trips_df.iterrows():
        for j, trip_j in trips_df.iterrows():
            # Skip same trip
            if i == j:
                continue
                
            # Check if trip_j can follow trip_i (time compatibility)
            if pd.isnull(trip_i['scheduled_end_time']) or pd.isnull(trip_j['scheduled_start_time']):
                continue
            
            # Get distance between end of trip_i and start of trip_j
            from_loc = trip_i['end_location']
            to_loc = trip_j['start_location']
            
            if from_loc == 'elmst' and to_loc == 'welst':
                distance = DISTANCES['elmst_to_welst']
            elif from_loc == 'welst' and to_loc == 'elmst':
                distance = DISTANCES['welst_to_elmst']
            elif from_loc == to_loc:
                distance = 0.1  # Short distance if same location
            else:
                # Use average for any other combinations
                distance = (DISTANCES['elmst_to_welst'] + DISTANCES['welst_to_elmst']) / 2
            
            # Calculate travel time
            travel_time = calculate_travel_time(distance)
            
            # Calculate earliest possible arrival time at start of trip_j
            earliest_arrival = trip_i['scheduled_end_time'] + timedelta(minutes=travel_time)
            
            # Check if trip_j can be served after trip_i
            if earliest_arrival <= trip_j['scheduled_start_time']:
                # Calculate waiting time
                waiting_time = (trip_j['scheduled_start_time'] - earliest_arrival).total_seconds() / 60
                
                # Calculate delay risk (if trip_i has history of delays)
                if not pd.isnull(trip_i['actual_end_time']) and not pd.isnull(trip_i['scheduled_end_time']):
                    delay = (trip_i['actual_end_time'] - trip_i['scheduled_end_time']).total_seconds() / 60
                    delay_risk = max(0, delay)
                else:
                    delay_risk = 0
                
                # Calculate costs
                deadheading_cost = distance * cost_per_mile + travel_time * cost_per_minute
                waiting_cost = waiting_time * waiting_cost_factor * cost_per_minute
                delay_risk_cost = delay_risk * delay_risk_factor * cost_per_minute
                total_cost = deadheading_cost + waiting_cost + delay_risk_cost
                
                connection = {
                    'from_id': trip_i['half_trip_id'],
                    'to_id': trip_j['half_trip_id'],
                    'from_type': 'trip',
                    'to_type': 'trip',
                    'from_location': trip_i['end_location'],
                    'to_location': trip_j['start_location'],
                    'distance_miles': distance,
                    'travel_time_minutes': travel_time,
                    'waiting_time_minutes': waiting_time,
                    'deadheading_cost': deadheading_cost,
                    'waiting_cost': waiting_cost,
                    'delay_risk_cost': delay_risk_cost,
                    'total_cost': total_cost
                }
                connections.append(connection)
    
    # Convert to DataFrame
    connections_df = pd.DataFrame(connections)
    return connections_df

def solve_arc_flow_mdvsp(trips_df, connections_df, depot_id=DEPOT_ID, max_vehicles=10):
    """
    Solve the MDVSP using the arc-flow formulation with Gurobi
    
    Args:
        trips_df: DataFrame of trip nodes
        connections_df: DataFrame of connections between trips
        depot_id: ID of the depot
        max_vehicles: Maximum number of vehicles available
        
    Returns:
        Solution DataFrame with vehicle schedules
    """
    print("Solving MDVSP using arc-flow formulation with Gurobi...")
    start_time = time.time()
    
    # Create a list of all trips
    trips = trips_df['half_trip_id'].tolist()
    
    # Create a dictionary to store connections
    connections = {}
    for _, conn in connections_df.iterrows():
        from_id = conn['from_id']
        to_id = conn['to_id']
        total_cost = conn['total_cost']
        
        # Add connection to the dictionary
        if from_id not in connections:
            connections[from_id] = {}
        connections[from_id][to_id] = total_cost
    
    # Create the Gurobi model
    model = gp.Model("MDVSP_ArcFlow")
    
    # Create variables
    x = {}
    for _, conn in connections_df.iterrows():
        from_id = conn['from_id']
        to_id = conn['to_id']
        
        # Create binary variable for this connection
        x[(from_id, to_id)] = model.addVar(vtype=GRB.BINARY, name=f"x_{from_id}_{to_id}")
    
    # Update the model to include new variables
    model.update()
    
    # Objective function: minimize total cost
    obj = gp.LinExpr()
    for _, conn in connections_df.iterrows():
        from_id = conn['from_id']
        to_id = conn['to_id']
        obj += x[(from_id, to_id)] * conn['total_cost']
    
    model.setObjective(obj, GRB.MINIMIZE)
    
    # Constraint 1: Flow conservation for trips
    for trip in trips:
        # Inflow equals outflow for each trip
        inflow = gp.LinExpr()
        outflow = gp.LinExpr()
        
        for from_id in connections:
            if trip in connections[from_id]:
                inflow += x[(from_id, trip)]
        
        if trip in connections:
            for to_id in connections[trip]:
                outflow += x[(trip, to_id)]
        
        model.addConstr(inflow == 1, f"Trip_{trip}_must_be_served")
        model.addConstr(inflow == outflow, f"Flow_conservation_{trip}")
    
    # Constraint 2: Maximum number of vehicles
    outflow_from_depot = gp.LinExpr()
    if depot_id in connections:
        for to_id in connections[depot_id]:
            outflow_from_depot += x[(depot_id, to_id)]
    
    model.addConstr(outflow_from_depot <= max_vehicles, "Max_vehicles")
    
    # Set time limit and solver parameters
    model.setParam('TimeLimit', 300)  # 5 minutes time limit
    model.setParam('OutputFlag', 1)   # Display solver output
    
    # Solve the model
    model.optimize()
    
    end_time = time.time()
    solution_time = end_time - start_time
    
    print(f"Status: {model.status}")
    if model.status == GRB.OPTIMAL or model.status == GRB.TIME_LIMIT:
        print(f"Total cost: {model.objVal}")
    print(f"Solution time: {solution_time:.2f} seconds")
    
    # Extract solution
    if model.status == GRB.OPTIMAL or model.status == GRB.TIME_LIMIT:
        # Create a list to store vehicle schedules
        vehicle_schedules = []
        
        # Start with connections from depot
        if depot_id in connections:
            for to_id in connections[depot_id]:
                if x[(depot_id, to_id)].x > 0.5:  # If this connection is used
                    # Start a new vehicle schedule
                    schedule = [{'type': 'depot', 'id': depot_id, 'location': depot_id}]
                    
                    # Follow the path of this vehicle
                    current_id = to_id
                    while current_id != depot_id:
                        # Add this trip to the schedule
                        trip_info = trips_df[trips_df['half_trip_id'] == current_id].iloc[0].to_dict()
                        schedule.append({
                            'type': 'trip',
                            'id': current_id,
                            'start_location': trip_info['start_location'],
                            'end_location': trip_info['end_location'],
                            'start_time': trip_info['scheduled_start_time'],
                            'end_time': trip_info['scheduled_end_time'],
                            'route_id': trip_info['route_id'],
                            'direction_id': trip_info['direction_id']
                        })
                        
                        # Find the next trip/depot
                        next_id = None
                        if current_id in connections:
                            for possible_next in connections[current_id]:
                                if x[(current_id, possible_next)].x > 0.5:
                                    next_id = possible_next
                                    break
                        
                        if next_id is None:
                            print(f"Warning: No next trip found for {current_id}")
                            break
                            
                        current_id = next_id
                    
                    # Close the schedule with return to depot
                    schedule.append({'type': 'depot', 'id': depot_id, 'location': depot_id})
                    
                    # Add this schedule to the list
                    vehicle_schedules.append(schedule)
        
        # Convert to DataFrame for easier analysis
        schedule_rows = []
        for vehicle_id, schedule in enumerate(vehicle_schedules):
            for stop_num, stop in enumerate(schedule):
                row = {
                    'vehicle_id': vehicle_id + 1,
                    'stop_num': stop_num + 1,
                    'type': stop['type'],
                    'id': stop['id']
                }
                
                # Add location information
                if 'location' in stop:
                    row['location'] = stop['location']
                
                # Add trip details if this is a trip
                if stop['type'] == 'trip':
                    row.update({
                        'start_location': stop['start_location'],
                        'end_location': stop['end_location'],
                        'start_time': stop['start_time'],
                        'end_time': stop['end_time'],
                        'route_id': stop['route_id'],
                        'direction_id': stop['direction_id']
                    })
                
                schedule_rows.append(row)
        
        schedules_df = pd.DataFrame(schedule_rows)
        return schedules_df
    else:
        print("No optimal solution found.")
        return None

def solve_set_partitioning_mdvsp(trips_df, connections_df, depot_id='fellsway_garage', max_vehicles=10):
    """
    Solve the MDVSP using the set partitioning formulation with Gurobi
    
    Args:
        trips_df: DataFrame of trip nodes
        connections_df: DataFrame of connections between trips
        depot_id: ID of the depot
        max_vehicles: Maximum number of vehicles available
        
    Returns:
        Solution DataFrame with vehicle schedules
    """
    print("Solving MDVSP using set partitioning formulation with Gurobi...")
    start_time = time.time()
    
    # Create a list of all trips
    trips = trips_df['half_trip_id'].tolist()
    
    # Create a graph representation for the network
    graph = {}
    for _, conn in connections_df.iterrows():
        from_id = conn['from_id']
        to_id = conn['to_id']
        cost = conn['total_cost']
        
        if from_id not in graph:
            graph[from_id] = {}
        graph[from_id][to_id] = cost
    
    # Generate all feasible vehicle schedules (paths from depot to depot)
    print("Generating feasible vehicle schedules...")
    feasible_schedules = []
    
    # Use a recursive function to find all paths from depot to depot
    def find_paths(current_path, visited_trips, current_cost):
        current_node = current_path[-1]
        
        # If we've returned to the depot and visited at least one trip, we have a feasible schedule
        if current_node == depot_id and len(current_path) > 2:
            # Extract just the trip IDs (exclude depot at start and end)
            trip_ids = [node for node in current_path[1:-1]]
            feasible_schedules.append({
                'trips': trip_ids,
                'cost': current_cost,
                'path': current_path.copy()  # Keep full path for reference
            })
            return
        
        # If we're at the maximum path length, stop recursion
        if len(current_path) > len(trips) + 2:
            return
        
        # Try all possible next nodes
        if current_node in graph:
            for next_node, edge_cost in graph[current_node].items():
                # Skip if this trip has already been visited
                if next_node in trips and next_node in visited_trips:
                    continue
                
                # Add this node to the path
                new_path = current_path + [next_node]
                new_visited = visited_trips.copy()
                if next_node in trips:
                    new_visited.add(next_node)
                
                # Continue recursion
                find_paths(new_path, new_visited, current_cost + edge_cost)
    
    # Start with paths from the depot
    find_paths([depot_id], set(), 0)
    
    print(f"Generated {len(feasible_schedules)} feasible schedules")
    
    # Create the Gurobi model
    model = gp.Model("MDVSP_SetPartitioning")
    
    # Create variables - one for each feasible schedule
    y = {}
    for i, schedule in enumerate(feasible_schedules):
        y[i] = model.addVar(vtype=GRB.BINARY, name=f"y_{i}")
    
    # Update model to incorporate new variables
    model.update()
    
    # Objective function: minimize total cost
    obj = gp.LinExpr()
    for i, schedule in enumerate(feasible_schedules):
        obj += y[i] * schedule['cost']
    
    model.setObjective(obj, GRB.MINIMIZE)
    
    # Constraint 1: Each trip must be covered exactly once
    for trip in trips:
        constr = gp.LinExpr()
        for i, schedule in enumerate(feasible_schedules):
            if trip in schedule['trips']:
                constr += y[i]
        model.addConstr(constr == 1, f"Trip_{trip}_must_be_served")
    
    # Constraint 2: Maximum number of vehicles
    vehicles_constr = gp.LinExpr()
    for i in range(len(feasible_schedules)):
        vehicles_constr += y[i]
    model.addConstr(vehicles_constr <= max_vehicles, "Max_vehicles")
    
    # Set time limit and solver parameters
    model.setParam('TimeLimit', 300)  # 5 minutes time limit
    model.setParam('OutputFlag', 1)   # Display solver output
    
    # Solve the model
    model.optimize()
    
    end_time = time.time()
    solution_time = end_time - start_time
    
    print(f"Status: {model.status}")
    if model.status == GRB.OPTIMAL or model.status == GRB.TIME_LIMIT:
        print(f"Total cost: {model.objVal}")
    print(f"Solution time: {solution_time:.2f} seconds")
    
    # Extract solution
    if model.status == GRB.OPTIMAL or model.status == GRB.TIME_LIMIT:
        # Create a list to store vehicle schedules
        schedule_rows = []
        
        # Process each selected schedule
        vehicle_id = 1
        for i, schedule in enumerate(feasible_schedules):
            if y[i].x > 0.5:  # If this schedule is used
                path = schedule['path']
                
                # Add depot departure
                schedule_rows.append({
                    'vehicle_id': vehicle_id,
                    'stop_num': 1,
                    'type': 'depot',
                    'id': depot_id,
                    'location': depot_id
                })
                
                # Add each trip in the schedule
                for stop_num, trip_id in enumerate(schedule['trips']):
                    # Get trip details
                    trip_info = trips_df[trips_df['half_trip_id'] == trip_id].iloc[0].to_dict()
                    
                    schedule_rows.append({
                        'vehicle_id': vehicle_id,
                        'stop_num': stop_num + 2,
                        'type': 'trip',
                        'id': trip_id,
                        'start_location': trip_info['start_location'],
                        'end_location': trip_info['end_location'],
                        'start_time': trip_info['scheduled_start_time'],
                        'end_time': trip_info['scheduled_end_time'],
                        'route_id': trip_info['route_id'],
                        'direction_id': trip_info['direction_id']
                    })
                
                # Add depot return
                schedule_rows.append({
                    'vehicle_id': vehicle_id,
                    'stop_num': len(schedule['trips']) + 2,
                    'type': 'depot',
                    'id': depot_id,
                    'location': depot_id
                })
                
                vehicle_id += 1
        
        schedules_df = pd.DataFrame(schedule_rows)
        return schedules_df
    else:
        print("No optimal solution found.")
        return None

def analyze_lp_relaxation_set_partitioning(trips_df, connections_df, depot_id='fellsway_garage', max_vehicles=10):
    """
    Analyze the LP relaxation of set partitioning formulation using Gurobi
    
    Args:
        trips_df: DataFrame of trip nodes
        connections_df: DataFrame of connections between trips
        depot_id: ID of the depot
        max_vehicles: Maximum number of vehicles available
    """
    print("Analyzing LP relaxation of set partitioning formulation with Gurobi...")
    start_time = time.time()
    
    # Create a list of all trips
    trips = trips_df['half_trip_id'].tolist()
    
    # Create a graph representation for the network
    graph = {}
    for _, conn in connections_df.iterrows():
        from_id = conn['from_id']
        to_id = conn['to_id']
        cost = conn['total_cost']
        
        if from_id not in graph:
            graph[from_id] = {}
        graph[from_id][to_id] = cost
    
    # Generate all feasible vehicle schedules
    feasible_schedules = []
    
    # Use a recursive function to find all paths from depot to depot
    def find_paths(current_path, visited_trips, current_cost):
        current_node = current_path[-1]
        
        # If we've returned to the depot and visited at least one trip, we have a feasible schedule
        if current_node == depot_id and len(current_path) > 2:
            # Extract just the trip IDs (exclude depot at start and end)
            trip_ids = [node for node in current_path[1:-1]]
            feasible_schedules.append({
                'trips': trip_ids,
                'cost': current_cost
            })
            return
        
        # If we're at the maximum path length, stop recursion
        if len(current_path) > len(trips) + 2:
            return
        
        # Try all possible next nodes
        if current_node in graph:
            for next_node, edge_cost in graph[current_node].items():
                # Skip if this trip has already been visited
                if next_node in trips and next_node in visited_trips:
                    continue
                
                # Add this node to the path
                new_path = current_path + [next_node]
                new_visited = visited_trips.copy()
                if next_node in trips:
                    new_visited.add(next_node)
                
                # Continue recursion
                find_paths(new_path, new_visited, current_cost + edge_cost)
    
    # Start with paths from the depot
    find_paths([depot_id], set(), 0)
    
    print(f"Generated {len(feasible_schedules)} feasible schedules")
    
    # Create the Gurobi model
    model = gp.Model("MDVSP_SetPartitioning_LP")
    
    # Create continuous variables (for LP relaxation)
    y = {}
    for i, schedule in enumerate(feasible_schedules):
        y[i] = model.addVar(lb=0, ub=1, vtype=GRB.CONTINUOUS, name=f"y_{i}")
    
    # Update model to incorporate new variables
    model.update()
    
    # Objective function: minimize total cost
    obj = gp.LinExpr()
    for i, schedule in enumerate(feasible_schedules):
        obj += y[i] * schedule['cost']
    
    model.setObjective(obj, GRB.MINIMIZE)
    
    # Constraint 1: Each trip must be covered exactly once
    for trip in trips:
        constr = gp.LinExpr()
        for i, schedule in enumerate(feasible_schedules):
            if trip in schedule['trips']:
                constr += y[i]
        model.addConstr(constr == 1, f"Trip_{trip}_must_be_served")
    
    # Constraint 2: Maximum number of vehicles
    vehicles_constr = gp.LinExpr()
    for i in range(len(feasible_schedules)):
        vehicles_constr += y[i]
    model.addConstr(vehicles_constr <= max_vehicles, "Max_vehicles")
    
    # Set solver parameters
    model.setParam('OutputFlag', 0)  # Suppress solver output
    
    # Solve the LP relaxation
    model.optimize()
    
    end_time = time.time()
    solution_time = end_time - start_time
    
    print(f"Set partitioning LP relaxation - Status: {model.status}")
    if model.status == GRB.OPTIMAL:
        print(f"Set partitioning LP relaxation - Objective: {model.objVal}")
    print(f"Solution time: {solution_time:.2f} seconds")

def analyze_lp_relaxation(trips_df, connections_df, depot_id=DEPOT_ID, max_vehicles=10):
    """
    Analyze the LP relaxation of arc-flow formulation using Gurobi
    
    Args:
        trips_df: DataFrame of trip nodes
        connections_df: DataFrame of connections between trips
        depot_id: ID of the depot
        max_vehicles: Maximum number of vehicles available
    """
    print("Analyzing LP relaxation of arc-flow formulation with Gurobi...")
    start_time = time.time()
    
    # Create a list of all trips
    trips = trips_df['half_trip_id'].tolist()
    
    # Create a dictionary to store connections
    connections = {}
    for _, conn in connections_df.iterrows():
        from_id = conn['from_id']
        to_id = conn['to_id']
        total_cost = conn['total_cost']
        
        # Add connection to the dictionary
        if from_id not in connections:
            connections[from_id] = {}
        connections[from_id][to_id] = total_cost
    
    # Create the Gurobi model for arc-flow
    model = gp.Model("MDVSP_ArcFlow_LP")
    
    # Create continuous variables
    x = {}
    for _, conn in connections_df.iterrows():
        from_id = conn['from_id']
        to_id = conn['to_id']
        
        
        # Create continuous variable for this connection
        x[(from_id, to_id)] = model.addVar(lb=0, ub=1, vtype=GRB.CONTINUOUS, name=f"x_{from_id}_{to_id}")
    
    # Update model to incorporate new variables
    model.update()
    
    # Objective function: minimize total cost
    obj = gp.LinExpr()
    for _, conn in connections_df.iterrows():
        from_id = conn['from_id']
        to_id = conn['to_id']
        obj += x[(from_id, to_id)] * conn['total_cost']
    
    model.setObjective(obj, GRB.MINIMIZE)
    
    # Constraint 1: Flow conservation for trips
    for trip in trips:
        # Inflow equals outflow for each trip
        inflow = gp.LinExpr()
        outflow = gp.LinExpr()
        
        for from_id in connections:
            if trip in connections[from_id]:
                inflow += x[(from_id, trip)]
        
        if trip in connections:
            for to_id in connections[trip]:
                outflow += x[(trip, to_id)]
        
        model.addConstr(inflow == 1, f"Trip_{trip}_must_be_served")
        model.addConstr(inflow == outflow, f"Flow_conservation_{trip}")
    
    # Constraint 2: Maximum number of vehicles
    outflow_from_depot = gp.LinExpr()
    if depot_id in connections:
        for to_id in connections[depot_id]:
            outflow_from_depot += x[(depot_id, to_id)]
    
    model.addConstr(outflow_from_depot <= max_vehicles, "Max_vehicles")
    
    # Set solver parameters
    model.setParam('OutputFlag', 0)  # Suppress solver output
    
    # Solve the LP relaxation
    model.optimize()
    
    end_time = time.time()
    solution_time = end_time - start_time
    
    print(f"Arc-flow LP relaxation - Status: {model.status}")
    if model.status == GRB.OPTIMAL:
        print(f"Arc-flow LP relaxation - Objective: {model.objVal}")
    print(f"Solution time: {solution_time:.2f} seconds")

def main():
    # Input and output paths
    input_csv = 'filtered_route100_15h_20h.csv'
    trip_nodes_output = 'route100_15h_20h_trip_nodes_gurobi.csv'
    connections_output = 'route100_15h_20h_connections_gurobi.csv'
    arc_flow_solution_output = 'route100_15h_20h_arc_flow_solution_gurobi.csv'
    set_partitioning_solution_output = 'route100_15h_20h_set_partitioning_solution_gurobi.csv'
    
    try:
        # Step 1: Create trip nodes
        print("\n--- STEP 1: Creating trip nodes ---")
        trips_df = create_trip_nodes(input_csv)
        
        # Print summary
        print(f"\nCreated {len(trips_df)} trip nodes")
        if not trips_df.empty:
            print("\nSample trip nodes:")
            print(trips_df.head())
            
            # Save to CSV
            trips_df.to_csv(trip_nodes_output, index=False)
            print(f"\nTrip nodes saved to {trip_nodes_output}")
        
        # Step 2: Calculate connections and costs
        print("\n--- STEP 2: Calculating connections and costs ---")
        connections_df = calculate_costs(trips_df)
        
        # Print summary
        print(f"\nCreated {len(connections_df)} connections")
        print("\nSample connections:")
        print(connections_df.head())
        
        # Save to CSV
        connections_df.to_csv(connections_output, index=False)
        print(f"\nConnections saved to {connections_output}")
        
        # Step 3: Analyze LP relaxation of arc-flow formulation
        print("\n--- STEP 3: Analyzing LP relaxation of arc-flow formulation ---")
        analyze_lp_relaxation(trips_df, connections_df)
        
        # Step 4: Analyze LP relaxation of set partitioning formulation
        print("\n--- STEP 4: Analyzing LP relaxation of set partitioning formulation ---")
        analyze_lp_relaxation_set_partitioning(trips_df, connections_df)
        
        # Step 5: Solve using arc-flow formulation
        print("\n--- STEP 5: Solving with arc-flow formulation ---")
        arc_flow_solution = solve_arc_flow_mdvsp(trips_df, connections_df)
        
        if arc_flow_solution is not None:
            print("\nSample arc-flow solution:")
            print(arc_flow_solution.head())
            
            # Save to CSV
            arc_flow_solution.to_csv(arc_flow_solution_output, index=False)
            print(f"\nArc-flow solution saved to {arc_flow_solution_output}")
        
        # Step 6: Solve using set partitioning formulation
        print("\n--- STEP 6: Solving with set partitioning formulation ---")
        set_partitioning_solution = solve_set_partitioning_mdvsp(trips_df, connections_df)
        
        if set_partitioning_solution is not None:
            print("\nSample set partitioning solution:")
            print(set_partitioning_solution.head())
            
            # Save to CSV
            set_partitioning_solution.to_csv(set_partitioning_solution_output, index=False)
            print(f"\nSet partitioning solution saved to {set_partitioning_solution_output}")
    
    except Exception as e:
        print(f"Error: {e}")

if __name__ == "__main__":
    main()


--- STEP 1: Creating trip nodes ---
Total records: 48
Unique half_trip_ids: 24
Unique stop_ids: [52711  8301  8302 52720]
Unique time_point_ids: ['welst' 'elm']

Created 24 trip nodes

Sample trip nodes:
   half_trip_id  route_id direction_id start_location end_location  \
0    65802625.0       100     Outbound          welst          elm   
1    65802626.0       100      Inbound            elm        welst   
2    65802627.0       100     Outbound          welst          elm   
3    65802628.0       100      Inbound            elm        welst   
4    65802629.0       100     Outbound          welst          elm   

  scheduled_start_time  scheduled_end_time   actual_start_time  \
0  2025-04-29 15:00:00 2025-04-29 15:11:00 2025-04-29 15:06:11   
1  2025-04-29 15:15:00 2025-04-29 15:26:00 2025-04-29 15:24:30   
2  2025-04-29 15:30:00 2025-04-29 15:41:00 2025-04-29 15:38:30   
3  2025-04-29 15:45:00 2025-04-29 15:57:00 2025-04-29 15:48:49   
4  2025-04-29 16:00:00 2025-04-29 16:11:00 2

/var/folders/v8/1793d44577q9lqgx_01s8skm0000gn/T/ipykernel_19708/4176640161.py:88: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  trips_df[col] = pd.to_datetime(trips_df[col], errors='coerce')
/var/folders/v8/1793d44577q9lqgx_01s8skm0000gn/T/ipykernel_19708/4176640161.py:88: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  trips_df[col] = pd.to_datetime(trips_df[col], errors='coerce')
/var/folders/v8/1793d44577q9lqgx_01s8skm0000gn/T/ipykernel_19708/4176640161.py:88: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  trips_df[col] = pd.to_datetime(trips_df[col], errors='coerce')
/var

Generated 16777215 feasible schedules
Set partitioning LP relaxation - Status: 11
Solution time: 1524.13 seconds

--- STEP 5: Solving with arc-flow formulation ---
Solving MDVSP using arc-flow formulation with Gurobi...
Set parameter TimeLimit to value 300
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[arm] - Darwin 23.6.0 23G93)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 49 rows, 324 columns and 924 nonzeros
Model fingerprint: 0x17dbffab
Variable types: 0 continuous, 324 integer (324 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [7e+00, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Found heuristic solution: objective 701.6166667
Presolve removed 12 rows and 58 columns
Presolve time: 0.01s
Presolved: 37 rows, 266 columns, 510 nonzeros
Found heuristic solution: objective 446.3833333
Variable types: 0 continuous, 266 integer (266 binary